In [ ]:
import numpy as np
import math as math
import pandas as pd
from numpy import random
from random import seed
from random import randint
import itertools
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.io  
from scipy.stats import expon
import random
from IPython.display import clear_output
from collections import deque
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, Reshape
from tensorflow.keras.optimizers import Adam
from __future__ import print_function
import sys
import itertools 
from collections import deque

from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, Reshape
from tensorflow.keras.optimizers import Adam
from scipy.stats import truncnorm

In [ ]:
def Loc_Fun(RSU_T,UAV_T,HAP_T,LEO_T,RSU_r,UAV_r,HAP_r,LEO_r,LEO_alt,HAP_alt,UAV_alt): 

    
    while True:
        RSU_loc=np.zeros((2, RSU_T))
        UAV_loc=np.zeros((3, UAV_T))
        HAP_loc=np.zeros((3, HAP_T))
        LEO_loc=np.zeros((3, LEO_T))
        
        R_UAV_asign= np.zeros((RSU_T,UAV_T))
        R_HAP_asign= np.zeros((RSU_T,HAP_T))
        R_LEO_asign= np.zeros((RSU_T,LEO_T))
        U_HAP_asign= np.zeros((UAV_T,HAP_T))
        U_LEO_asign= np.zeros((UAV_T,LEO_T))
        H_LEO_asign= np.zeros((HAP_T,LEO_T))
        
        RSU_loc[0][0]=RSU_r/2
        RSU_loc[1][0]=5
        for i in range(RSU_T-1):
            i=i+1
            rn=np.random.randint(int(RSU_r/2), 1.5*RSU_r, size=(1,1))


            RSU_loc[0][i]=RSU_loc[0][i-1]+rn
            RSU_loc[1][i]=5 


        UAV_loc[0][0]=UAV_r/2
        UAV_loc[2][0]=UAV_alt 
        for i in range(UAV_T-1):
            i=i+1
            rn=np.random.randint(int(UAV_r), 2*UAV_r, size=(1,1))
            UAV_loc[0][i]=UAV_loc[0][i-1]+rn
            UAV_loc[1][i]=0
            UAV_loc[2][i]=UAV_alt 


        HAP_loc[0][0]=HAP_r/2
        HAP_loc[2][0]=HAP_alt 
        for i in range(HAP_T-1):
            i=i+1
            rn=np.random.randint(int(HAP_r), 2*HAP_r, size=(1,1))
            HAP_loc[0][i]=HAP_loc[0][i-1]+rn
            HAP_loc[1][i]=0
            HAP_loc[2][i]=HAP_alt 
            
        LEO_loc[0][0]=LEO_r
        LEO_loc[2][0]=LEO_alt 
        for i in range(LEO_T-1):
            i=i+1
            LEO_loc[0][i]=LEO_r
            LEO_loc[1][i]=0
            LEO_loc[2][i]=LEO_alt 
#%%-----------------------------Loc Para End-------------------------------------------------------------------------%%

        for r in range(RSU_T):
            for l in range(UAV_T):
                if ((((UAV_loc[1][l] - RSU_loc[1][r])**2) + ((UAV_loc[0][l]-RSU_loc[0][r])**2)) <= (UAV_r)**2):
                    R_UAV_asign[r][l] = 1

        for r in range(RSU_T):
            for h in range(HAP_T):
                if ((((HAP_loc[1][h]- RSU_loc[1][r] )**2) + ((HAP_loc[0][h]-RSU_loc[0][r])**2)) <= (HAP_r)**2):
                    R_HAP_asign[r][h] = 1
                 
        for r in range(RSU_T):
            for s in range(LEO_T):
                if ((((LEO_loc[1][s]- RSU_loc[1][r] )**2) + ((LEO_loc[0][s]-RSU_loc[0][r])**2)) <= (LEO_r)**2):
                    R_LEO_asign[r][s] = 1
#%%--------------------------------------------------------------------------------------------%%

        for l in range(UAV_T):
            for h in range(HAP_T):
                if((HAP_loc[0][h]-HAP_r) < 0):
                    if (((UAV_loc[0][l]-UAV_r)> (HAP_loc[0][h]-HAP_r))and((UAV_loc[0][l]+UAV_r)**2< (HAP_loc[0][h]+HAP_r)**2)):
                        U_HAP_asign[l][h] = 1
                else:
                    if (((UAV_loc[0][l]-UAV_r)**2> (HAP_loc[0][h]-HAP_r)**2)and((UAV_loc[0][l]+UAV_r)**2< (HAP_loc[0][h]+HAP_r)**2)):
                        U_HAP_asign[l][h] = 1

        for l in range(UAV_T):
            for s in range(LEO_T):
                if((LEO_loc[0][s]-LEO_r) < 0):
                    if (((UAV_loc[0][l]-UAV_r)> (LEO_loc[0][s]-LEO_r))and((UAV_loc[0][l]+UAV_r)**2< (LEO_loc[0][s]+LEO_r)**2)):
                        U_LEO_asign[l][s] = 1
                else: 
                    if (((UAV_loc[0][l]-UAV_r)**2> (LEO_loc[0][s]-LEO_r)**2)and((UAV_loc[0][l]+UAV_r)**2< (LEO_loc[0][s]+LEO_r)**2)):
                        U_LEO_asign[l][s] = 1
#%%--------------------------------------------------------------------------------------------%%                    
        for h in range(HAP_T):
            for s in range(LEO_T):
                if((LEO_loc[0][s]-LEO_r) < 0):
                    if (((HAP_loc[0][h]-HAP_r)> (LEO_loc[0][s]-LEO_r))and((HAP_loc[0][h]+HAP_r)**2< (LEO_loc[0][s]+LEO_r)**2)):
                        H_LEO_asign[h][s] = 1
                else:
                    if (((HAP_loc[0][h]-HAP_r)**2> (LEO_loc[0][s]-LEO_r)**2)and((HAP_loc[0][h]+HAP_r)**2< (LEO_loc[0][s]+LEO_r)**2)):
                        H_LEO_asign[h][s] = 1
#%%--------------------------------------------------------------------------------------------%%               

        ru_sum=np.zeros((1, RSU_T))
        rh_sum=np.zeros((1, RSU_T))
        rs_sum=np.zeros((1, RSU_T))
        uh_sum=np.zeros((1, UAV_T))
        us_sum=np.zeros((1, UAV_T))
        hs_sum=np.zeros((1, HAP_T))
        for r in range(RSU_T):
            ru_sum[0][r]=sum(R_UAV_asign[r])
            rh_sum[0][r]=sum(R_HAP_asign[r])
            rs_sum[0][r]=sum(R_LEO_asign[r])
        for l in range(UAV_T):
            uh_sum[0][l]=sum(U_HAP_asign[l])
            us_sum[0][l]=sum(U_LEO_asign[l])
            
        for h in range(HAP_T):
            hs_sum[0][h]=sum(H_LEO_asign[h])
            
        if ((np.all(ru_sum >0)) and (np.all(rh_sum >0)) and (np.all(rs_sum >0)) and ((np.all(uh_sum >0))) and ((np.all(us_sum >0))) and ((np.all(hs_sum >0)))):
            break        
        
    return RSU_loc, UAV_loc, HAP_loc, LEO_loc

In [ ]:
def VN_EN_Assign(IP,V_spd):
    out=[]
    V=IP[0]
    RSU_T=IP[1]
    RSU_loc=IP[2]
    RSU_r=IP[3]
    UAV_T=IP[4]
    UAV_loc=IP[5]
    UAV_r=IP[6]
    HAP_T=IP[7]
    HAP_loc=IP[8]
    HAP_r=IP[9]
    LEO_T=IP[10]
    LEO_loc=IP[11]
    LEO_r=IP[12]
    V_RSU_asign= np.zeros((V, RSU_T))
    V_RSU_Soj= np.zeros((V, RSU_T))
    V_RSU_dist= np.zeros((V, RSU_T))
    V_UAV_asign= np.zeros((V, UAV_T))
    V_UAV_Soj= np.zeros((V, UAV_T))
    V_UAV_dist= np.zeros((V, UAV_T))
    V_HAP_asign= np.zeros((V, HAP_T))
    V_HAP_Soj= np.zeros((V, HAP_T))
    V_HAP_dist= np.zeros((V, HAP_T))
    V_LEO_asign= np.zeros((V, LEO_T))
    V_LEO_Soj= np.zeros((V, LEO_T))
    V_LEO_dist= np.zeros((V, LEO_T))
    
    R_UAV_asign= np.zeros((RSU_T,UAV_T))
    R_HAP_asign= np.zeros((RSU_T,HAP_T))
    U_HAP_asign= np.zeros((UAV_T,HAP_T))
    R_UAV_dist= np.zeros((RSU_T, UAV_T))
    R_HAP_dist= np.zeros((RSU_T, HAP_T))
    U_HAP_dist= np.zeros((UAV_T, HAP_T))
    V_RSU_d= np.zeros((V, RSU_T))
    V_UAV_d= np.zeros((V, UAV_T))
    V_HAP_d= np.zeros((V,HAP_T))
    V_LEO_d= np.zeros((V,LEO_T))
    RSU_SC=np.zeros((1, V))
    VN_loc=np.zeros((3, V))
    while True:
        for i in range(V):
            VN_loc[2][i]=random.choice([1,2,3,4,5,6])
            while True:
                VN_loc[0][i]=np.random.randint(RSU_loc[0][0], RSU_loc[0][RSU_T-1], size=(1,1))
                VN_loc[1][i]=0
                for j in range(RSU_T):

                    if RSU_loc[0][j] >= VN_loc[0][i] and (RSU_r)**(2) > (RSU_loc[1][j] - VN_loc[1][i])**(2):

                        if ((RSU_loc[0][j]-(math.sqrt((RSU_r)**(2) - (RSU_loc[1][j] - VN_loc[1][i])**(2)))) <= VN_loc[0][i] and VN_loc[0][i] <= (RSU_loc[0][j]+(math.sqrt((RSU_r)**(2) - (RSU_loc[1][j] - VN_loc[1][i])**(2)))) ):
                            V_RSU_asign[i][j] = 1
                            V_RSU_d[i][j] = (math.sqrt((RSU_r)**(2) - (RSU_loc[1][j] - VN_loc[1][i])**(2))+(RSU_loc[0][j]-VN_loc[0][i]))
                            V_RSU_Soj[i][j] = ((math.sqrt((RSU_r)**(2) - (RSU_loc[1][j] - VN_loc[1][i])**(2))+(RSU_loc[0][j]-VN_loc[0][i]))/V_spd[i])
                            V_RSU_dist[i][j] = math.hypot(RSU_loc[0][j]  - VN_loc[0][i], RSU_loc[1][j]  - VN_loc[1][i])
                        else:
                            V_RSU_asign[i][j]=0
                            V_RSU_d[i][j] =0
                            V_RSU_Soj[i][j]=0
                            V_RSU_dist[i][j]=0
                    elif RSU_loc[0][j] < VN_loc[0][i] and (RSU_r)**(2) > (RSU_loc[1][j] - VN_loc[1][i])**(2):

                        if ((RSU_loc[0][j]-(math.sqrt((RSU_r)**(2) - (RSU_loc[1][j] - VN_loc[1][i])**(2)))) <= VN_loc[0][i] and VN_loc[0][i] <= (RSU_loc[0][j]+(math.sqrt((RSU_r)**(2) - (RSU_loc[1][j] - VN_loc[1][i])**(2))))  ):
                            V_RSU_asign[i][j] = 1
                            V_RSU_d[i][j] = (math.sqrt((RSU_r)**(2) - (RSU_loc[1][j] - VN_loc[1][i])**(2))-(VN_loc[0][i] - RSU_loc[0][j]))
                            V_RSU_Soj[i][j] = ((math.sqrt((RSU_r)**(2) - (RSU_loc[1][j] - VN_loc[1][i])**(2))-(VN_loc[0][i] - RSU_loc[0][j]))/V_spd[i])
                            V_RSU_dist[i][j] =  math.hypot(RSU_loc[0][j]  - VN_loc[0][i], RSU_loc[1][j]  - VN_loc[1][i])
                        else:
                            V_RSU_asign[i][j]=0
                            V_RSU_d[i][j] =0
                            V_RSU_Soj[i][j]=0
                            V_RSU_dist[i][j]=0
                for j in range(RSU_T):
                    if (V_RSU_Soj[i][j]==0):
                        V_RSU_asign[i][j]=0
                        V_RSU_dist[i][j]=0
                if  ((sum(V_RSU_asign[i][:]) <4)): 
                    break  

            a_c = V_RSU_asign[i]
            ls = [ii for ii, e in enumerate(a_c) if e != 0]
            RSU_SC[0][i]=len(ls)



        for i in range(V):
            for j in range(UAV_T):
                if (((UAV_loc[0][j]+UAV_r) >= VN_loc[0][i]) and (VN_loc[0][i] > (UAV_loc[0][j]-UAV_r))):
                    V_UAV_asign[i][j] = 1
                    V_UAV_d[i][j] = (UAV_loc[0][j] + (UAV_r-VN_loc[0][i]))
                    V_UAV_Soj[i][j] = (V_UAV_d[i][j]/V_spd[i])
                    V_UAV_dist[i][j] =math.sqrt((UAV_loc[0][j] - VN_loc[0][i])**2 + (UAV_loc[1][j] - VN_loc[1][i])**2 + (UAV_loc[2][j] - 0)**2)


        for i in range(V):
            for j in range(HAP_T):
                if (((HAP_loc[0][j]+HAP_r) >= VN_loc[0][i]) and (VN_loc[0][i] > (HAP_loc[0][j]-HAP_r))):
                    V_HAP_asign[i][j] = 1
                    V_HAP_d[i][j] = (HAP_loc[0][j] + (HAP_r-VN_loc[0][i]))
                    V_HAP_Soj[i][j] = (V_HAP_d[i][j]/V_spd[i])  
                    V_HAP_dist[i][j] = math.sqrt((HAP_loc[0][j] - VN_loc[0][i])**2 + (HAP_loc[1][j] - VN_loc[1][i])**2 + (HAP_loc[2][j] - 0)**2)

        for i in range(V):
            for j in range(LEO_T):
                if (((LEO_loc[0][j]+LEO_r) >= VN_loc[0][i]) and (VN_loc[0][i] > (LEO_loc[0][j]-LEO_r))):
                    V_LEO_asign[i][j] = 1
                    V_LEO_d[i][j] = (LEO_loc[0][j] + (LEO_r-VN_loc[0][i]))
                    V_LEO_Soj[i][j] = (V_LEO_d[i][j]/V_spd[i])  
                    V_LEO_dist[i][j] = math.sqrt((LEO_loc[0][j] - VN_loc[0][i])**2 + (LEO_loc[1][j] - VN_loc[1][i])**2 + (LEO_loc[2][j] - 0)**2)
 
                    
                    
        vr_sum=np.zeros((1, V))
        vu_sum=np.zeros((1, V))
        vh_sum=np.zeros((1, V))
        vs_sum=np.zeros((1, V))
        for v in range(V):
            vr_sum[0][v]=sum(V_RSU_asign[v])
            vu_sum[0][v]=sum(V_UAV_asign[v])
            vh_sum[0][v]=sum(V_HAP_asign[v])
            vs_sum[0][v]=sum(V_LEO_asign[v])

        if ((np.all(vr_sum >0)) and (np.all(vu_sum >0)) and (np.all(vh_sum >0)) and (np.all(vs_sum >0))):
            break

    out.append(VN_loc)
    out.append(V_RSU_asign)
    out.append(V_RSU_Soj)
    out.append(V_RSU_dist)
    out.append(V_UAV_asign)
    out.append(V_UAV_Soj)
    out.append(V_UAV_dist)
    out.append(V_HAP_asign)
    out.append(V_HAP_Soj)
    out.append(V_HAP_dist)
    out.append(V_LEO_asign)
    out.append(V_LEO_Soj)
    out.append(V_LEO_dist)
    out.append(V_RSU_d)
    out.append(V_UAV_d)
    out.append(V_HAP_d)
    out.append(V_LEO_d)
    return out

In [ ]:
def Chn_Capacity(N1N2_dist, BN2, PN1, b_0, theta):
    N0dbm = -45  # noise power in dBm
    N0 = 10 ** ((N0dbm - 30) / 10)  # noise power in Watts
    lam = (3 * (10 ** 8)) / BN2
    PL = 20 * math.log10(N1N2_dist / (10 ** 3)) + 92.45 + 20 * math.log10(BN2 / (10 ** 9))
    path_loss = 10 ** ((-PL) / 10)


    channel_gain = b_0 * (N1N2_dist ** theta) * path_loss

    # Calculate the Shannon channel capacity
    C = BN2 * math.log2(1 + (PN1 * channel_gain) / N0)

    return C



In [ ]:
#-----------Edit-----------------------------
def Task_Proc_Main(IP, CR, DR_R_R, CU, DR_R_U, CH, DR_R_H, CL, DR_R_S,g1,g2, decisions):
    
#============Task Processing===============================================
    out=[]
    V=IP[0]
    RSU_T=IP[1]
    UAV_T=IP[4]
    HAP_T=IP[7]
    LEO_T=IP[10]
    TS=IP[31]
    TSD=IP[32]
    psi_dmp=IP[33]
    Pcomp_m=IP[34]
    Ptp_v=IP[35]
    Pcomp_r=IP[36]
    Ptp_r=IP[37]
    Pcomp_u=IP[38]
    Ptp_u=IP[39]
    Pcomp_h=IP[40]
    Ptp_h=IP[41]
    Pcomp_s=IP[42]
    Ptp_s=IP[43]
    Cm=IP[44]
    Soj_T_VR=IP[45]
    Soj_T_VU=IP[46]
    Soj_T_VH=IP[47]
    Soj_T_VS=IP[48]
    gamma_1=IP[49]
    gamma_2=IP[50]
    Ser_RSU=IP[51]
    Ser_UAV=IP[52]
    Ser_HAP=IP[53]
    Ser_LEO=IP[54]
    V_loc=IP[55]


    #--------Local Device Computation----------------------------

    TPcomp_v=np.zeros((1, (V)))
    EPcomp_v=np.zeros((1, (V)))
    for v in range(V):
        TPcomp_v[0][v]=((TS[0][v]*psi_dmp)/(Cm[0][v]))
        EPcomp_v[0][v]=(Pcomp_m[0][v]*TPcomp_v[0][v])

        
    #-----------Offloading Time and Energy------------------
    TPcomp_r=np.zeros((V, RSU_T))
    EPcomp_r=np.zeros((V, RSU_T))
    TPcomp_u=np.zeros((V, UAV_T))
    EPcomp_u=np.zeros((V, UAV_T))
    TPcomp_h=np.zeros((V, HAP_T))
    EPcomp_h=np.zeros((V, HAP_T))
    TPcomp_s=np.zeros((V, LEO_T))
    EPcomp_s=np.zeros((V, LEO_T))
    
    TPcomU_vr=np.zeros((V, RSU_T))
    EPcomU_vr=np.zeros((V, RSU_T))
    TPcomD_vr=np.zeros((V, RSU_T))
    EPcomD_vr=np.zeros((V, RSU_T))
    TPcomU_rv=np.zeros((V, RSU_T))
    EPcomU_rv=np.zeros((V, RSU_T))
    TPcomD_rv=np.zeros((V, RSU_T))
    EPcomD_rv=np.zeros((V, RSU_T))
    
    TPcomU_vu=np.zeros((V, UAV_T))
    EPcomU_vu=np.zeros((V, UAV_T))
    TPcomD_vu=np.zeros((V, UAV_T))
    EPcomD_vu=np.zeros((V, UAV_T))
    TPcomU_uv=np.zeros((V, UAV_T))
    EPcomU_uv=np.zeros((V, UAV_T))
    TPcomD_uv=np.zeros((V, UAV_T))
    EPcomD_uv=np.zeros((V, UAV_T))
    
    TPcomU_vh=np.zeros((V, HAP_T))
    EPcomU_vh=np.zeros((V, HAP_T))
    TPcomD_vh=np.zeros((V, HAP_T))
    EPcomD_vh=np.zeros((V, HAP_T))
    TPcomU_hv=np.zeros((V, HAP_T))
    EPcomU_hv=np.zeros((V, HAP_T))
    TPcomD_hv=np.zeros((V, HAP_T))
    EPcomD_hv=np.zeros((V, HAP_T))
    

    
    TPcomU_vs=np.zeros((V, LEO_T))
    EPcomU_vs=np.zeros((V, LEO_T))
    TPcomD_vs=np.zeros((V, LEO_T))
    EPcomD_vs=np.zeros((V, LEO_T))
    TPcomU_sv=np.zeros((V, LEO_T))
    EPcomU_sv=np.zeros((V, LEO_T))
    TPcomD_sv=np.zeros((V, LEO_T))
    EPcomD_sv=np.zeros((V, LEO_T))
    

    
    for v in range(V):
        if(decisions[v][0]==0):
            for r in range(RSU_T):
                if(decisions[v][1]==r):
                    TPcomp_r[v][r]=((TS[0][v]*psi_dmp)/(CR[v][r]))
                    EPcomp_r[v][r]=(Pcomp_r[0][r]*TPcomp_r[v][r])
                    TPcomU_vr[v][r]=((TS[0][v])/DR_R_R[v][r])
                    EPcomU_vr[v][r]=TPcomU_vr[v][r]*Ptp_v[0][v]
                    EPcomU_rv[v][r]=TPcomU_vr[v][r]*Ptp_r[0][r]
                    TPcomD_vr[v][r]=((TSD[0][v])/DR_R_R[v][r])
                    EPcomD_vr[v][r]=TPcomD_vr[v][r]*Ptp_v[0][v]
                    EPcomD_rv[v][r]=TPcomD_vr[v][r]*Ptp_r[0][r]
                    
                    
                    
    #------------Total Task Processing Time and Energy--------------\n    
    TP_offl_vr=np.zeros((V, RSU_T))
    EP_offl_vr=np.zeros((V, RSU_T))
    TP_offl_rv=np.zeros((V, RSU_T))
    EP_offl_rv=np.zeros((V, RSU_T))
    TP_offl_T_r=np.zeros((V, RSU_T))
    EP_offl_T_r=np.zeros((V, RSU_T))    

    TP_loc=np.zeros((1, (V)))
    EP_loc=np.zeros((1, (V)))
    for v in range(V):
        if(decisions[v][0]==0):
            for r in range(RSU_T):
                if(decisions[v][1]==r):
                    TP_offl_vr[v][r]=TPcomU_vr[v][r]  + TPcomD_vr[v][r]
                    EP_offl_vr[v][r]=EPcomU_vr[v][r]  + EPcomD_vr[v][r]
                    TP_offl_rv[v][r]=TPcomp_r[v][r]
                    EP_offl_rv[v][r]=EPcomp_r[v][r]
                    
                    TP_offl_T_r[v][r]=TP_offl_vr[v][r]  + TP_offl_rv[v][r]
                    EP_offl_T_r[v][r]=0.5*EP_offl_vr[v][r]  + 0.5*EP_offl_rv[v][r]

                    TP_loc[0][v]=TPcomp_v[0][v]
                    EP_loc[0][v]=0.5*EPcomp_v[0][v]                    

                
    for v in range(V):
        if(decisions[v][0]==1):
            for u in range(UAV_T):
                if(decisions[v][1]==u):
                    TPcomp_u[v][u]=((TS[0][v]*psi_dmp)/(CU[v][u]))
                    EPcomp_u[v][u]=(Pcomp_u[0][u]*TPcomp_u[v][u])
                    
                    TPcomU_vu[v][u]=((TS[0][v])/DR_R_U[v][u])
                    EPcomU_vu[v][u]=TPcomU_vu[v][u]*Ptp_v[0][v]
                    EPcomU_uv[v][u]=TPcomU_vu[v][u]*Ptp_u[0][u]
                    TPcomD_vu[v][u]=((TSD[0][v])/DR_R_U[v][u])
                    EPcomD_vu[v][u]=TPcomD_vu[v][u]*Ptp_v[0][v]
                    EPcomD_uv[v][u]=TPcomD_vu[v][u]*Ptp_u[0][u]
                    
                 
                    
    #------------Total Task Processing Time and Energy (UAV)--------------\n    
    TP_offl_vu=np.zeros((V, UAV_T))
    EP_offl_vu=np.zeros((V, UAV_T))
    TP_offl_uv=np.zeros((V, UAV_T))
    EP_offl_uv=np.zeros((V, UAV_T))
    TP_offl_T_u=np.zeros((V, UAV_T))
    EP_offl_T_u=np.zeros((V, UAV_T))    


    for v in range(V):
        if(decisions[v][0]==1):
            for u in range(UAV_T):
                if(decisions[v][1]==u):
                    TP_offl_vu[v][u]=TPcomU_vu[v][u]  + TPcomD_vu[v][u]
                    EP_offl_vu[v][u]=EPcomU_vu[v][u]  + EPcomD_vu[v][u]
                    TP_offl_uv[v][u]=TPcomp_u[v][u]
                    EP_offl_uv[v][u]=EPcomp_u[v][u]
                    
                    TP_offl_T_u[v][u]=TP_offl_vu[v][u]  + TP_offl_uv[v][u]
                    EP_offl_T_u[v][u]=0.5*EP_offl_vu[v][u]  + 0.5*EP_offl_uv[v][u]

                    TP_loc[0][v]=TPcomp_v[0][v]
                    EP_loc[0][v]=0.5*EPcomp_v[0][v]                    

                    
                
    for v in range(V):
        if(decisions[v][0]==2):
            for h in range(HAP_T):
                if(decisions[v][1]==h):
                    TPcomp_h[v][h]=((TS[0][v]*psi_dmp)/(CH[v][h]))
                    EPcomp_h[v][h]=(Pcomp_h[0][h]*TPcomp_h[v][h])
                    
                    TPcomU_vh[v][h]=((TS[0][v])/DR_R_H[v][h])
                    EPcomU_vh[v][h]=TPcomU_vh[v][h]*Ptp_v[0][v]
                    EPcomU_hv[v][h]=TPcomU_vh[v][h]*Ptp_h[0][h]
                    TPcomD_vh[v][h]=((TSD[0][v])/DR_R_H[v][h])
                    EPcomD_vh[v][h]=TPcomD_vh[v][h]*Ptp_v[0][v]
                    EPcomD_hv[v][h]=TPcomD_vh[v][h]*Ptp_h[0][h]
                    
    #------------Total Task Processing Time and Energy (HAP)--------------\n    
    TP_offl_vh=np.zeros((V, HAP_T))
    EP_offl_vh=np.zeros((V, HAP_T))
    TP_offl_hv=np.zeros((V, HAP_T))
    EP_offl_hv=np.zeros((V, HAP_T))
    TP_offl_T_h=np.zeros((V, HAP_T))
    EP_offl_T_h=np.zeros((V, HAP_T))    


    for v in range(V):
        if(decisions[v][0]==2):
            for h in range(HAP_T):
                if(decisions[v][1]==h):
                    TP_offl_vh[v][h]=TPcomU_vh[v][h]  + TPcomD_vh[v][h]
                    EP_offl_vh[v][h]=EPcomU_vh[v][h]  + EPcomD_vh[v][h]
                    TP_offl_hv[v][h]=TPcomp_h[v][h]
                    EP_offl_hv[v][h]=EPcomp_h[v][h]
                    
                    TP_offl_T_h[v][h]=TP_offl_vh[v][h]  + TP_offl_hv[v][h]
                    EP_offl_T_h[v][h]=0.5*EP_offl_vh[v][h]  + 0.5*EP_offl_hv[v][h]

                    TP_loc[0][v]=TPcomp_v[0][v]
                    EP_loc[0][v]=0.5*EPcomp_v[0][v]   
                    

    for v in range(V):
        if(decisions[v][0]==3):
            for s in range(LEO_T):
                if(decisions[v][1]==s):
                    TPcomp_s[v][s]=((TS[0][v]*psi_dmp)/(CL[v][s]))
                    EPcomp_s[v][s]=(Pcomp_s[0][s]*TPcomp_s[v][s])
                    
                    TPcomU_vs[v][s]=((TS[0][v])/DR_R_S[v][s])
                    EPcomU_vs[v][s]=TPcomU_vs[v][s]*Ptp_v[0][v]
                    EPcomU_sv[v][s]=TPcomU_vs[v][s]*Ptp_s[0][s]
                    TPcomD_vs[v][s]=((TSD[0][v])/DR_R_S[v][s])
                    EPcomD_vs[v][s]=TPcomD_vs[v][s]*Ptp_v[0][v]
                    EPcomD_sv[v][s]=TPcomD_vs[v][s]*Ptp_s[0][s]
                    
    
                    
    #------------Total Task Processing Time and Energy (HAP)--------------\n    
    TP_offl_vs=np.zeros((V, LEO_T))
    EP_offl_vs=np.zeros((V, LEO_T))
    TP_offl_sv=np.zeros((V, LEO_T))
    EP_offl_sv=np.zeros((V, LEO_T))
    TP_offl_T_s=np.zeros((V, LEO_T))
    EP_offl_T_s=np.zeros((V, LEO_T))    


    for v in range(V):
        if(decisions[v][0]==3):
            for s in range(LEO_T):
                if(decisions[v][1]==s):
                    TP_offl_vs[v][s]=TPcomU_vs[v][s]  + TPcomD_vs[v][s]
                    EP_offl_vs[v][s]=EPcomU_vs[v][s]  + EPcomD_vs[v][s]
                    TP_offl_sv[v][s]=TPcomp_s[v][s]
                    EP_offl_sv[v][s]=EPcomp_s[v][s]
                    
                    TP_offl_T_s[v][s]=TP_offl_vs[v][s]  + TP_offl_sv[v][s]
                    EP_offl_T_s[v][s]=0.5*EP_offl_vs[v][s]  + 0.5*EP_offl_sv[v][s]

                    TP_loc[0][v]=TPcomp_v[0][v]
                    EP_loc[0][v]=0.5*EPcomp_v[0][v]   
                    
                    
                    
                    
                    
    T_L=np.zeros((1,V))
    T_L_V=np.zeros((1,V))
    T_L_E=np.zeros((1,V))
    T_E=np.zeros((1,V))
    T_E_V=np.zeros((1,V))
    T_E_E=np.zeros((1,V))
    No_of_Soj_L_Fail=0
    No_of_Ser_H_Req=0
    
    for v in range(V):
        if(decisions[v][0]==0):
            for r in range(RSU_T):
                if(decisions[v][1]==r):
                    T_L[0][v]=max((decisions[v][2]*TP_offl_T_r[v][r]), ((1-decisions[v][2])*TP_loc[0][v]))
                    T_L_V[0][v]= (1-decisions[v][2])*TP_loc[0][v]
                    T_L_E[0][v]= (decisions[v][2]*TP_offl_T_r[v][r])
                    T_E[0][v]=((decisions[v][2]*EP_offl_T_r[v][r])+ ((1-decisions[v][2])*EP_loc[0][v]))
                    T_E_V[0][v]= (1-decisions[v][2])*EP_loc[0][v]
                    T_E_E[0][v]= (decisions[v][2]*EP_offl_T_r[v][r])
                    if(T_L_E[0][v]>Soj_T_VR[v][r]):
                        No_of_Soj_L_Fail=No_of_Soj_L_Fail+1
                    if V_loc[2][v] not in Ser_RSU[r]:
                        No_of_Ser_H_Req += 1
                        
    for v in range(V):
        if(decisions[v][0]==1):
            for u in range(UAV_T):
                if(decisions[v][1]==u):
                    T_L[0][v]=max((decisions[v][2]*TP_offl_T_u[v][u]), ((1-decisions[v][2])*TP_loc[0][v]))
                    T_E[0][v]=((decisions[v][2]*EP_offl_T_u[v][u])+ ((1-decisions[v][2])*EP_loc[0][v]))  
                    T_L_V[0][v]= (1-decisions[v][2])*TP_loc[0][v]
                    T_L_E[0][v]= (decisions[v][2]*TP_offl_T_u[v][u])
                    T_E_V[0][v]= (1-decisions[v][2])*EP_loc[0][v]
                    T_E_E[0][v]= (decisions[v][2]*EP_offl_T_u[v][u])
                    if(T_L_E[0][v]>Soj_T_VU[v][u]):
                        No_of_Soj_L_Fail=No_of_Soj_L_Fail+1
                    if V_loc[2][v] not in Ser_UAV[u]:
                        No_of_Ser_H_Req += 1
        
    for v in range(V):
        if(decisions[v][0]==2):
            for h in range(HAP_T):
                if(decisions[v][1]==h):
                    T_L[0][v]=max((decisions[v][2]*TP_offl_T_h[v][h]), ((1-decisions[v][2])*TP_loc[0][v]))
                    T_E[0][v]=((decisions[v][2]*EP_offl_T_h[v][h])+ ((1-decisions[v][2])*EP_loc[0][v]))  
                    T_L_V[0][v]= (1-decisions[v][2])*TP_loc[0][v]
                    T_L_E[0][v]= (decisions[v][2]*TP_offl_T_h[v][h])
                    T_E_V[0][v]= (1-decisions[v][2])*EP_loc[0][v]
                    T_E_E[0][v]= (decisions[v][2]*EP_offl_T_h[v][h])
                    if(T_L_E[0][v]>Soj_T_VH[v][h]):
                        No_of_Soj_L_Fail=No_of_Soj_L_Fail+1
                    if V_loc[2][v] not in Ser_HAP[h]:
                        No_of_Ser_H_Req += 1
                    
    for v in range(V):
        if(decisions[v][0]==3):
            for s in range(LEO_T):
                if(decisions[v][1]==s):
                    T_L[0][v]=max((decisions[v][2]*TP_offl_T_s[v][s]), ((1-decisions[v][2])*TP_loc[0][v]))
                    T_E[0][v]=((decisions[v][2]*EP_offl_T_s[v][s])+ ((1-decisions[v][2])*EP_loc[0][v]))  
                    T_L_V[0][v]= (1-decisions[v][2])*TP_loc[0][v]
                    T_L_E[0][v]= (decisions[v][2]*TP_offl_T_s[v][s])
                    T_E_V[0][v]= (1-decisions[v][2])*EP_loc[0][v]
                    T_E_E[0][v]= (decisions[v][2]*EP_offl_T_s[v][s])
                    if(T_L_E[0][v]>Soj_T_VS[v][s]):
                        No_of_Soj_L_Fail=No_of_Soj_L_Fail+1
                    if V_loc[2][v] not in Ser_LEO[s]:
                        No_of_Ser_H_Req += 1

    No_of_Ser_L_Fail=0
    No_of_Ser_H_Fail=0
    TC=np.zeros((1, V))
    
    for v in range(V):
        TC[0][v]= g1*T_L[0][v] + g2*T_E[0][v]
        if(T_L[0][v]>4):
            No_of_Ser_L_Fail=No_of_Ser_L_Fail+1


                    
    out.append(T_L)
    out.append(T_E)
    out.append(T_L_V)
    out.append(T_L_E)
    out.append(T_E_V)
    out.append(T_E_E)
    out.append(TP_loc)
    out.append(EP_loc)
    out.append(TC)
    out.append(No_of_Ser_L_Fail)
    out.append(No_of_Soj_L_Fail)
    out.append(No_of_Ser_H_Req)

    return out



In [ ]:
def Data_Rate(IP, decisions, RSU_B, UAV_B, HAP_B, LEO_B):
    
    V=IP[0]
    RSU_T=IP[1]
    UAV_T=IP[4]
    HAP_T=IP[7]
    LEO_T=IP[10]
    
    PNR=IP[21]
    PNU=IP[22]
    PNH=IP[23]
    PNL=IP[24]
    b_0=IP[25]
    theta=IP[26]
    VN_RSU_dist=IP[27]
    VN_UAV_dist=IP[28]
    VN_HAP_dist=IP[29]
    VN_LEO_dist=IP[30]

    
    
    
    Rate_VR=np.zeros((V,RSU_T))
    Rate_VU=np.zeros((V,UAV_T))
    Rate_VH=np.zeros((V,HAP_T))
    Rate_VL=np.zeros((V,LEO_T))
    

    for v in range(V):
        if(decisions[v][0]==0):
            for r in range(RSU_T):
                if(decisions[v][1]==r):
                    Rate_VR[v][r]=Chn_Capacity(VN_RSU_dist[v][r], RSU_B[v][r], PNR[0][r], b_0, theta)

    for v in range(V):
        if(decisions[v][0]==1):
            for u in range(UAV_T):
                if(decisions[v][1]==u):
                    Rate_VU[v][u]=Chn_Capacity(VN_UAV_dist[v][u], UAV_B[v][u], PNU[0][u], b_0, theta)

    for v in range(V):
        if(decisions[v][0]==2):
            for h in range(HAP_T):
                if(decisions[v][1]==h):
                    Rate_VH[v][h]=Chn_Capacity(VN_HAP_dist[v][h], HAP_B[v][h], PNH[0][h], b_0, theta)

    for v in range(V):
        if(decisions[v][0]==3):
            for s in range(LEO_T):
                if(decisions[v][1]==s):
                    Rate_VL[v][s]=Chn_Capacity(VN_LEO_dist[v][s], LEO_B[v][s], PNL[0][s], b_0, theta)
                    
    return Rate_VR, Rate_VU, Rate_VH, Rate_VL

In [ ]:

def Resource_Allocation( IP,decisions, NOdes_assign_R, NOdes_assign_U, NOdes_assign_H, NOdes_assign_L):
    out=[]
    
    V=IP[0]
    RSU_T=IP[1]
    UAV_T=IP[4]
    HAP_T=IP[7]
    LEO_T=IP[10]
    RSU_B=IP[13]
    UAV_B=IP[14]
    HAP_B=IP[15]
    LEO_B=IP[16]
    RSU_C=IP[17]
    UAV_C=IP[18]
    HAP_C=IP[19]
    LEO_C=IP[20]

    
    
    RSU_B_Assign=np.zeros((V,RSU_T))
    UAV_B_Assign=np.zeros((V,UAV_T))
    HAP_B_Assign=np.zeros((V,HAP_T))
    LEO_B_Assign=np.zeros((V,LEO_T))
    RSU_C_Assign=np.zeros((V,RSU_T))
    UAV_C_Assign=np.zeros((V,UAV_T))
    HAP_C_Assign=np.zeros((V,HAP_T))
    LEO_C_Assign=np.zeros((V,LEO_T))
    for v in range(V):
        if(decisions[v][0]==0):
            for r in range(RSU_T): 
                if(decisions[v][1]==r):
                    RSU_B_Assign[v][r]=RSU_B[0][r]/NOdes_assign_R[r]
                    RSU_C_Assign[v][r]=RSU_C[0][r]/NOdes_assign_R[r]

    for v in range(V):
        if(decisions[v][0]==1):
            for u in range(UAV_T): 
                if(decisions[v][1]==u):
                    UAV_B_Assign[v][u]=UAV_B[0][u]/NOdes_assign_U[u]
                    UAV_C_Assign[v][u]=UAV_C[0][u]/NOdes_assign_U[u]

    for v in range(V):
        if(decisions[v][0]==2):
            for h in range(HAP_T): 
                if(decisions[v][1]==h):
                    HAP_B_Assign[v][h]=HAP_B[0][h]/NOdes_assign_H[h]
                    HAP_C_Assign[v][h]=HAP_C[0][h]/NOdes_assign_H[h]
                    
    for v in range(V):
        if(decisions[v][0]==3):
            for s in range(LEO_T): 
                if(decisions[v][1]==s):
                    LEO_B_Assign[v][s]=LEO_B[0][s]/NOdes_assign_L[s]
                    LEO_C_Assign[v][s]=LEO_C[0][s]/NOdes_assign_L[s]
    out.append(RSU_B_Assign)
    out.append(UAV_B_Assign)
    out.append(HAP_B_Assign)
    out.append(LEO_B_Assign)
    out.append(RSU_C_Assign)
    out.append(UAV_C_Assign)
    out.append(HAP_C_Assign)
    out.append(LEO_C_Assign)
    
                    
    return out

In [ ]:


class Agent:
    def __init__(self, optimizer, state_size, action_size, gamma, epsilon):
        self._state_size = state_size
        self._action_size = action_size
        self._optimizer = optimizer
        self.experience_replay = deque(maxlen=50000)
        self.gamma = gamma
        self.epsilon = epsilon
        
        # Build networks
        self.q_network = self._build_compile_model()
        self.target_network = self._build_compile_model()
        
        # Build models with an input shape to ensure they are initialized
        self.q_network.build((None, 1))
        self.target_network.build((None, 1))
        
        # Initialize with dummy weights to avoid errors
        self._initialize_dummy_weights()
        
        self.alighn_target_model()

    def store(self, state, action_id, reward, next_state):
        self.experience_replay.append((state, action_id, reward, next_state))
        
    def _build_compile_model(self):
        model = Sequential()
        model.add(Embedding(input_dim=self._state_size, output_dim=10))
        model.add(Reshape((10,)))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(self._action_size, activation='linear'))
        
        model.compile(loss='mse', optimizer=self._optimizer)
        return model

    def _initialize_dummy_weights(self):
        dummy_weights = [np.zeros_like(weight) for weight in self.q_network.get_weights()]
        self.q_network.set_weights(dummy_weights)
        self.target_network.set_weights(dummy_weights)

    def alighn_target_model(self):
        self.target_network.set_weights(self.q_network.get_weights())
        
    def act(self, state, q_values):
        if np.random.rand() <= self.epsilon:
            q_values = self.q_network.predict(state)
        return np.argmax(q_values[0])

    def retrain(self, batch_size):
        minibatch = random.sample(self.experience_replay, batch_size)
        for state, action, reward, next_state in minibatch: 
            target = self.q_network.predict(state)  
            t = self.target_network.predict(next_state)
            target[0][action] = reward + self.gamma * np.amax(t)
            self.q_network.fit(state, target, epochs=1, verbose=0)

    def load(self, name1, name2):
        self.q_network.load_weights(name1)
        self.target_network.load_weights(name2)

    def save(self, name1, name2):
        self.q_network.save_weights(name1)
        self.target_network.save_weights(name2)




In [ ]:
# class Agent:
#     def __init__(self, optimizer, state_size, action_size, gamma, epsilon):
#         self._state_size = state_size
#         self._action_size = action_size
#         self._optimizer = optimizer
#         self.experience_replay = deque(maxlen=50000)
#         self.gamma = gamma
#         self.epsilon = epsilon
        
#         # Build networks
#         self.q_network = self._build_compile_model()
#         self.target_network = self._build_compile_model()
#         self.alighn_target_model()

#     def store(self, state, action_id, reward, next_state):
#         self.experience_replay.append((state, action_id, reward, next_state))
        
#     def _build_compile_model(self):
#         model = Sequential()
#         model.add(Embedding(self._state_size, 10))
#         model.add(Reshape((10,)))
#         model.add(Dense(50, activation='relu'))
#         model.add(Dense(50, activation='relu'))
#         model.add(Dense(self._action_size, activation='linear'))
        
#         model.compile(loss='mse', optimizer=self._optimizer)
#         return model


    
#     def alighn_target_model(self):
#         self.target_network.set_weights(self.q_network.get_weights())
        
#     def act(self, state,q_values):
#         if np.random.rand() <= self.epsilon:
#             q_values = self.q_network.predict(state)
#             #print('q_values',q_values)
#         return np.argmax(q_values[0])


#     def retrain(self, batch_size):
#         minibatch = random.sample(self.experience_replay, batch_size)
#         for state, action, reward, next_state in minibatch: 
#             target = self.q_network.predict(state)  
            

#             t = self.target_network.predict(next_state)
            

#             target[0][action] = reward + self.gamma * np.amax(t)
            
#             #print('target[0][action]', target[0][action])
#             self.q_network.fit(state, target, epochs=1, verbose=0)




    
#     def load(self, name1, name2):
#         self.q_network.load_weights(name1)
#         self.target_network.load_weights(name2)


#     def save(self, name1, name2):
#         self.q_network.save_weights(name1)
#         self.target_network.save_weights(name2)


In [ ]:
gamma_1=0.5
gamma_2=0.5
RSU_r=50
UAV_r=300
HAP_r=1000
LEO_r=3000
RSU_T=int((1)*LEO_r/RSU_r) #Total RSU
UAV_T=int((1)*LEO_r/UAV_r) #Total UAV
HAP_T=int((1.5)*LEO_r/HAP_r) #Total HAP
LEO_T=1

B_RSU=(20*10**6)*np.ones((1, RSU_T))
B_UAV=(20*10**6)*np.ones((1, UAV_T))
B_HAP=(50*10**6)*np.ones((1, HAP_T))
B_LEO=(100*10**6)*np.ones((1, LEO_T))
C_RSU=(20*10**9)*np.ones((1, RSU_T))
C_UAV=(20*10**9)*np.ones((1, UAV_T))
C_HAP=(40*10**9)*np.ones((1, HAP_T))
C_LEO=(60*10**9)*np.ones((1, LEO_T))
UAV_alt=1000
HAP_alt=100000
LEO_alt=200000
services = [1, 2, 3, 4, 5 ,6]
services_per_RSU=3
services_per_UAV=3
services_per_HAP=4
services_per_LEO=6

service_allocation_RSU = {}
service_allocation_UAV = {}
service_allocation_HAP = {}
service_allocation_LEO = {}

for node_id in range(RSU_T):
    allocated_services = random.sample(services, services_per_RSU)
    service_allocation_RSU[node_id] = allocated_services
    
for node_id in range(UAV_T):
    allocated_services = random.sample(services, services_per_UAV)
    service_allocation_UAV[node_id] = allocated_services

for node_id in range(HAP_T):
    allocated_services = random.sample(services, services_per_HAP)
    service_allocation_HAP[node_id] = allocated_services

for node_id in range(LEO_T):
    allocated_services = random.sample(services, services_per_LEO)
    service_allocation_LEO[node_id] = allocated_services

RSU_loc, UAV_loc, HAP_loc, LEO_loc=Loc_Fun(RSU_T,UAV_T,HAP_T,LEO_T,RSU_r,UAV_r,HAP_r,LEO_r,LEO_alt,HAP_alt,UAV_alt) #---define locations---
Pcomp_r=(1.3)*np.ones((1, RSU_T))
Pcomp_u=(1.2)*np.ones((1, UAV_T))
Pcomp_h=(1.1)*np.ones((1, HAP_T))
Pcomp_s=(1.1)*np.ones((1, LEO_T))
P_tx_r=(1.3)*np.ones((1, RSU_T))
P_tx_u=(1.2)*np.ones((1, UAV_T))
P_tx_h=(1.1)*np.ones((1, HAP_T))
P_tx_s=(1.1)*np.ones((1, LEO_T))
P_rx_r=(1.1)*np.ones((1, RSU_T))
P_rx_u=(1)*np.ones((1, UAV_T))
P_rx_h=(0.9)*np.ones((1, HAP_T))
P_rx_s=(0.9)*np.ones((1, LEO_T))
b_0=1
theta=3

V=200
No_of_Ser=6
C_V=(8*10**9)*np.ones((1, V))
Pcomp_m=(1.3)*np.ones((1, V))
psi_dmp=1000
TS=(5*8*10**6)*np.ones((1, V))
TSD=(1*8*10**6)*np.ones((1, V))
DL_Req=4
P_tx_v=(1.5)*np.ones((1, V))
P_rx_v=(1.3)*np.ones((1, V))
P_comv=(1)*np.ones((1, V))


VN_spd=np.zeros(V)
min_spd = 8   #
max_spd = 14  
mean = 12     
sigma = 2    

a, b = (min_spd - mean) / sigma, (max_spd - mean) / sigma
trunc_dist = truncnorm(a, b, loc=mean, scale=sigma)

for v in range(V):
    VN_spd[v] = trunc_dist.rvs(size=1)[0]

VN_Main_IP=[]
VN_Main_IP.append(V)
VN_Main_IP.append(RSU_T)
VN_Main_IP.append(RSU_loc)
VN_Main_IP.append(RSU_r)
VN_Main_IP.append(UAV_T)
VN_Main_IP.append(UAV_loc)
VN_Main_IP.append(UAV_r)
VN_Main_IP.append(HAP_T)
VN_Main_IP.append(HAP_loc)
VN_Main_IP.append(HAP_r)
VN_Main_IP.append(LEO_T)
VN_Main_IP.append(LEO_loc)
VN_Main_IP.append(LEO_r)
VN_Main_IP.append(B_RSU)
VN_Main_IP.append(B_UAV)
VN_Main_IP.append(B_HAP)
VN_Main_IP.append(B_LEO)
VN_Main_IP.append(C_RSU)
VN_Main_IP.append(C_UAV)
VN_Main_IP.append(C_HAP)
VN_Main_IP.append(C_LEO)
VN_Main_IP.append(P_tx_r)
VN_Main_IP.append(P_tx_u)
VN_Main_IP.append(P_tx_h)
VN_Main_IP.append(P_tx_s)
VN_Main_IP.append(b_0)
VN_Main_IP.append(theta)




VN_assign_OUT =VN_EN_Assign(VN_Main_IP, VN_spd)

VN_loc=VN_assign_OUT[0]
VN_RSU_asign=VN_assign_OUT[1]
VN_RSU_Soj=VN_assign_OUT[2]
VN_RSU_dist=VN_assign_OUT[3]
VN_UAV_asign=VN_assign_OUT[4]
VN_UAV_Soj=VN_assign_OUT[5]
VN_UAV_dist=VN_assign_OUT[6]
VN_HAP_asign=VN_assign_OUT[7]
VN_HAP_Soj=VN_assign_OUT[8]
VN_HAP_dist=VN_assign_OUT[9]
VN_LEO_asign=VN_assign_OUT[10]
VN_LEO_Soj=VN_assign_OUT[11]
VN_LEO_dist=VN_assign_OUT[12]

VN_Main_IP.append(VN_RSU_dist)
VN_Main_IP.append(VN_UAV_dist)
VN_Main_IP.append(VN_HAP_dist)
VN_Main_IP.append(VN_LEO_dist)
VN_Main_IP.append(TS)
VN_Main_IP.append(TSD)
VN_Main_IP.append(psi_dmp)
VN_Main_IP.append(Pcomp_m)
VN_Main_IP.append(P_tx_v)
VN_Main_IP.append(Pcomp_r)
VN_Main_IP.append(P_tx_r)
VN_Main_IP.append(Pcomp_u)
VN_Main_IP.append(P_tx_u)
VN_Main_IP.append(Pcomp_h)
VN_Main_IP.append(P_tx_h)
VN_Main_IP.append(Pcomp_s)
VN_Main_IP.append(P_tx_s)
VN_Main_IP.append(C_V)
VN_Main_IP.append(VN_RSU_Soj)
VN_Main_IP.append(VN_UAV_Soj)
VN_Main_IP.append(VN_HAP_Soj)
VN_Main_IP.append(VN_LEO_Soj)
VN_Main_IP.append(gamma_1)
VN_Main_IP.append(gamma_2)
VN_Main_IP.append(service_allocation_RSU)
VN_Main_IP.append(service_allocation_UAV)
VN_Main_IP.append(service_allocation_HAP)
VN_Main_IP.append(service_allocation_LEO)
VN_Main_IP.append(VN_loc)

#--------Random Approach--------------------------
#1) Layer Selection
Layer_Selection_random=np.zeros(V)
Node_selection_random=np.zeros(V)
Offloading_random=np.zeros(V)
Random_decisions=np.zeros((V,3))

for v in range(V):
    Random_decisions[v][0] = random.choice([0, 1, 2, 3])
    if(Random_decisions[v][0] ==0):
        Node_seelction_choice=VN_RSU_asign[v]
        indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
        Random_decisions[v][1] = random.choice(indices_with_ones.tolist())
        Random_decisions[v][2]=random.random()
        
    if(Random_decisions[v][0] ==1):
        Node_seelction_choice=VN_UAV_asign[v]
        indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
        Random_decisions[v][1] = random.choice(indices_with_ones.tolist())
        Random_decisions[v][2]=random.random()

    if(Random_decisions[v][0] ==2):
        Node_seelction_choice=VN_HAP_asign[v]
        indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
        Random_decisions[v][1] = random.choice(indices_with_ones.tolist())
        Random_decisions[v][2]=random.random()
        
    if(Random_decisions[v][0] ==3):
        Node_seelction_choice=VN_LEO_asign[v]
        indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
        Random_decisions[v][1] = random.choice(indices_with_ones.tolist())
        Random_decisions[v][2]=random.random()

NO_VUs_RSU=np.zeros(RSU_T)
NO_VUs_UAV=np.zeros(UAV_T)
NO_VUs_HAP=np.zeros(HAP_T)
NO_VUs_LEO=np.zeros(LEO_T)
for v in range(V):
    if(Random_decisions[v][0]==0):
        for r in range(RSU_T):
            if(Random_decisions[v][1]==r):
                NO_VUs_RSU[r]=NO_VUs_RSU[r]+1

    if(Random_decisions[v][0]==1):
        for u in range(UAV_T):
            if(Random_decisions[v][1]==u):
                NO_VUs_UAV[u]=NO_VUs_UAV[u]+1

    if(Random_decisions[v][0]==2):
        for h in range(HAP_T):
            if(Random_decisions[v][1]==h):
                NO_VUs_HAP[h]=NO_VUs_HAP[h]+1
                
    if(Random_decisions[v][0]==3):
        for s in range(LEO_T):
            if(Random_decisions[v][1]==s):
                NO_VUs_LEO[s]=NO_VUs_LEO[s]+1

#-----Resource Allocation------------
RA_Random_OUT= Resource_Allocation( VN_Main_IP,  Random_decisions, NO_VUs_RSU, NO_VUs_UAV, NO_VUs_HAP, NO_VUs_LEO)

RSU_B_RA=RA_Random_OUT[0]
UAV_B_RA=RA_Random_OUT[1]
HAP_B_RA=RA_Random_OUT[2]
LEO_B_RA=RA_Random_OUT[3]
RSU_C_RA=RA_Random_OUT[4]
UAV_C_RA=RA_Random_OUT[5]
HAP_C_RA=RA_Random_OUT[6]
LEO_C_RA=RA_Random_OUT[7]






Rate_VR_R, Rate_VU_R, Rate_VH_R, Rate_VL_R= Data_Rate(VN_Main_IP,Random_decisions, RSU_B_RA, UAV_B_RA, HAP_B_RA, LEO_B_RA)
    
Cost=Task_Proc_Main(VN_Main_IP, RSU_C_RA, Rate_VR_R, UAV_C_RA, Rate_VU_R, HAP_C_RA, Rate_VH_R, LEO_C_RA, Rate_VL_R,0.5,0.5, Random_decisions)
T_L_R=Cost[0]
T_E_R=Cost[1]
T_L_V_R=Cost[2]
T_L_E_R=Cost[3]
T_E_V_R=Cost[3]
T_E_E_R=Cost[3]






In [ ]:
service_allocation_UAV[1]

In [ ]:
State_Space_H={}
H_S_D=[0,1,2,3]
H_S_CV=[0, 1, 2]
M_S=[0, 1, 2, 3]
P_SL=[0, 1, 2, 3]
k=0

State_Space_H = np.zeros((len(H_S_D) * len(H_S_CV) * len(M_S) * len(P_SL), 4))
k = 0

for i1 in range(len(H_S_D)):
    for i2 in range(len(H_S_CV)):
        for i3 in range(len(M_S)):
            for i4 in range(len(P_SL)):
                state = np.zeros(4)
                state[0] = H_S_D[i1]
                state[1] = H_S_CV[i2]
                state[2] = M_S[i3]
                state[3] = P_SL[i4]
                State_Space_H[k] = state
                k += 1
        
Action_Space_H= [0, 1, 2, 3]

In [ ]:

def Next_State_H( a_h, Req_Ser, RSU_T, NO_VUs_RSU, Max_U_RSU, UAV_T, NO_VUs_UAV, Max_U_UAV, HAP_T, NO_VUs_HAP, Max_U_HAP, LEO_T, NO_VUs_LEO, Max_U_LEO):

    S_N=np.zeros(4)

    if(a_h==0):
        S_N[0]=0

        H_S_CV_R=0
        H_S_CV_R_A=0
  
        for r in range(RSU_T):
     
            H_S_CV_R = H_S_CV_R + (NO_VUs_RSU[r]/Max_U_RSU)

        H_S_CV_R_A=(H_S_CV_R/RSU_T)

        if(H_S_CV_R_A <= 0.5):
            H_S_CV_R_s=0
        elif(0.5 < H_S_CV_R_A <= 1):
            H_S_CV_R_s=1
        else:
            H_S_CV_R_s=2
        S_N[1]=H_S_CV_R_s


        S_N[2]=0

        nodes_with_service=0
        service_layer_prob_R=0
        nodes_with_service = sum(1 for services in service_allocation_RSU.values() if Req_Ser in services)
        service_layer_prob_R=(nodes_with_service/RSU_T)
        if(0 <= service_layer_prob_R <  0.25 ):
            S_N[3]=3
        elif(0.25 <= service_layer_prob_R < 0.5):
            S_N[3]=2
        elif(0.5 <= service_layer_prob_R < 0.75):
            S_N[3]=1
        else:
            S_N[3]=0

    #------------------------------------------------------------
    if(a_h==1):
        S_N[0]=1

        H_S_CV_U=0
        H_S_CV_U_A=0

        for r in range(UAV_T):
            H_S_CV_U = H_S_CV_U + (NO_VUs_UAV[r]/Max_U_UAV)

        H_S_CV_U_A=(H_S_CV_U/UAV_T)

        if(H_S_CV_U_A <= 0.5):
            H_S_CV_U_s=0
        elif(0.5 < H_S_CV_U_A <= 1):
            H_S_CV_U_s=1
        else:
            H_S_CV_U_s=2
        S_N[1]=H_S_CV_U_s


        S_N[2]=2

        nodes_with_service=0
        service_layer_prob_U=0
        nodes_with_service = sum(1 for services in service_allocation_UAV.values() if Req_Ser in services)
        service_layer_prob_U=(nodes_with_service/UAV_T)
        if(0 <= service_layer_prob_U < 0.25 ):
            S_N[3]=3
        elif(0.25 <= service_layer_prob_U < 0.5):
            S_N[3]=2
        elif(0.5 <= service_layer_prob_U < 0.75):
            S_N[3]=1
        else:
            S_N[3]=0


    #------------------------------------------------------------
    if(a_h==2):
        S_N[0]=2

        H_S_CV_H=0
        H_S_CV_H_A=0

        for r in range(HAP_T):
            H_S_CV_H = H_S_CV_H + (NO_VUs_HAP[r]/Max_U_HAP)

        H_S_CV_H_A=(H_S_CV_H/HAP_T)

        if(H_S_CV_H_A <= 0.5):
            H_S_CV_H_s=0
        elif(0.5 < H_S_CV_H_A <= 1):
            H_S_CV_H_s=1
        else:
            H_S_CV_H_s=2
        S_N[1]=H_S_CV_H_s


        S_N[2]=1

        nodes_with_service=0
        service_layer_prob_H=0
        nodes_with_service = sum(1 for services in service_allocation_HAP.values() if Req_Ser in services)
        service_layer_prob_H=(nodes_with_service/HAP_T)
        if(0 <= service_layer_prob_H < 0.25 ):
            S_N[3]=3
        elif(0.25 <= service_layer_prob_H < 0.5):
            S_N[3]=2
        elif(0.5 <= service_layer_prob_H < 0.75):
            S_N[3]=1
        else:
            S_N[3]=0

     #------------------------------------------------------------
    if(a_h==3):
        S_N[0]=3

        H_S_CV_L=0
        H_S_CV_L_A=0

        for r in range(LEO_T):
            H_S_CV_L = H_S_CV_L + (NO_VUs_LEO[r]/Max_U_LEO)

        H_S_CV_L_A=(H_S_CV_L/LEO_T)

        if(H_S_CV_L_A <= 0.5):
            H_S_CV_L_s=0
        elif(0.5 < H_S_CV_L_A <= 1):
            H_S_CV_L_s=1
        else:
            H_S_CV_L_s=2
        S_N[1]=H_S_CV_L_s


        S_N[2]=3

        nodes_with_service=0
        service_layer_prob_L=0
        nodes_with_service = sum(1 for services in service_allocation_LEO.values() if Req_Ser in services)
        service_layer_prob_L=(nodes_with_service/LEO_T)
        if(0 <= service_layer_prob_L < 0.25 ):
            S_N[3]=3
        elif(0.25 <= service_layer_prob_L < 0.5):
            S_N[3]=2
        elif(0.5 <= service_layer_prob_L < 0.75):
            S_N[3]=1
        else:
            S_N[3]=0
    indices = np.where((State_Space_H == S_N).all(axis=1))[0]
    row_index = indices[0]
    return S_N, row_index



In [ ]:
#define a function that determines if the specified location is a terminal state
def is_terminal_state(current_row_index, current_column_index, rewards):
    if rewards[current_row_index, current_column_index] != 0:
        return False
    else:
        return True

#define a function that will choose a random, non-terminal starting location
def get_starting_location(environment_rows):
  #get a random row and column index
    current_row_index = np.random.randint(environment_rows)
    #current_column_index = np.random.randint(environment_columns)
    # while is_terminal_state(current_row_index, current_column_index, rewards):
    #     current_row_index = np.random.randint(environment_rows)
    #     current_column_index = np.random.randint(environment_columns)
    return current_row_index

In [ ]:
def get_next_action(current_row_index, epsilon, Action_Space, q_values):
    if np.random.random() < epsilon:
        return np.argmax(q_values[current_row_index])
    else: #choose a random action
        return random.choice(Action_Space)

In [ ]:
def get_next_action_M(current_row_index, Action_id, epsilon, Action_Space, q_values):
    if np.random.random() < epsilon:
        q_values_local= q_values[:, Action_id]
        a_id=np.argmax(q_values_local[current_row_index])
        return Action_Space[a_id]
    else: #choose a random action
        return random.choice(Action_Space)

In [ ]:
State_Space_M={}
R_S_N=[0,1,2,3]
S_S_N=[0, 1]
ST_S_N=[0, 1]


State_Space_M = np.zeros((len(R_S_N) * len(S_S_N) * len(ST_S_N), 3))
k = 0
for i1 in range(len(R_S_N)):
    for i2 in range(len(S_S_N)):
        for i3 in range(len(ST_S_N)):
            state = np.zeros(3)
            state[0] = R_S_N[i1]
            state[1] = S_S_N[i2]
            state[2] = ST_S_N[i3]
            State_Space_M[k] = state
            k += 1
        

    


In [ ]:
def M_Action_Space( a_h, v_id, VN_RSU_asign, VN_UAV_asign, VN_HAP_asign, VN_LEO_asign):
    
    if(a_h==0):
        VN_EN_asign_local=VN_RSU_asign[v_id]
        non_zero_indices = np.nonzero(VN_EN_asign_local)[0]
        M_A = np.zeros((len(non_zero_indices), len(VN_EN_asign_local)))


        for i, index in enumerate(non_zero_indices):
            row = np.zeros_like(VN_EN_asign_local)
            row[index] = 1
            M_A[i] = row

    elif (a_h==1):
        VN_EN_asign_local=VN_UAV_asign[v_id]
        non_zero_indices = np.nonzero(VN_EN_asign_local)[0]
        M_A = np.zeros((len(non_zero_indices), len(VN_EN_asign_local)))


        for i, index in enumerate(non_zero_indices):
            row = np.zeros_like(VN_EN_asign_local)
            row[index] = 1
            M_A[i] = row

    elif (a_h==2):
        VN_EN_asign_local=VN_HAP_asign[v_id]
        non_zero_indices = np.nonzero(VN_EN_asign_local)[0]
        M_A = np.zeros((len(non_zero_indices), len(VN_EN_asign_local)))
   
        for i, index in enumerate(non_zero_indices):
            row = np.zeros_like(VN_EN_asign_local)
            row[index] = 1
            M_A[i] = row

    else:
        VN_EN_asign_local=VN_LEO_asign[v_id]
        non_zero_indices = np.nonzero(VN_EN_asign_local)[0]
        M_A = np.zeros((len(non_zero_indices), len(VN_EN_asign_local)))


        for i, index in enumerate(non_zero_indices):
            row = np.zeros_like(VN_EN_asign_local)
            row[index] = 1
            M_A[i] = row
    return M_A


In [ ]:


def Next_State_M(
    a_h, a_m, VU_speed, Req_Ser, NO_VUs_RSU, Max_U_RSU, service_allocation_RSU, VN_RSU_Soj, RSU_r,
    NO_VUs_UAV, Max_U_UAV, service_allocation_UAV, VN_UAV_Soj, UAV_r,
    NO_VUs_HAP, Max_U_HAP, service_allocation_HAP, VN_HAP_Soj, HAP_r,
    NO_VUs_LEO, Max_U_LEO, service_allocation_LEO, VN_LEO_Soj, LEO_r):


    N_S_M=np.zeros(3)
    non_zero_indices = np.nonzero(a_m)

    if(a_h==0):
 
        CV_EN=(NO_VUs_RSU[non_zero_indices]/Max_U_RSU)

        Service=service_allocation_RSU[non_zero_indices[0][0]]
        Soj_T=VN_RSU_Soj[non_zero_indices]
   
        cov=RSU_r
        spd=VU_speed

    elif(a_h==1):

        CV_EN=(NO_VUs_UAV[non_zero_indices]/Max_U_UAV)
        Service=service_allocation_UAV[non_zero_indices[0][0]]
        Soj_T=VN_UAV_Soj[non_zero_indices]
        cov=UAV_r
        spd=VU_speed

    elif(a_h==2):

        CV_EN=(NO_VUs_HAP[non_zero_indices]/Max_U_HAP)
        Service=service_allocation_HAP[non_zero_indices[0][0]]
        Soj_T=VN_HAP_Soj[non_zero_indices]
        cov=HAP_r
        spd=VU_speed

    elif(a_h==3):
   
        CV_EN=(NO_VUs_LEO[non_zero_indices]/Max_U_LEO)
        Service=service_allocation_LEO[non_zero_indices[0][0]]
        Soj_T=VN_LEO_Soj[non_zero_indices]
        cov=LEO_r
        spd=VU_speed
    #-------------------------------------------- 
  
    if(0 <= CV_EN < 0.25):
        N_S_M[0]=0
    elif(0.25 <= CV_EN < 0.5):
        N_S_M[0]=1

    elif(0.5 <= CV_EN < 0.75):
        N_S_M[0]=2

    else:
        N_S_M[0]=3

    #-----------------------------------------     
    if Req_Ser in Service:
        N_S_M[1]=1
    else:
        N_S_M[1]=0    
    #--------------------------------------------       
    if(0 <= Soj_T < (cov/VU_speed) ):
        N_S_M[2]=0
    else:
        N_S_M[2]=1

    #----------------------------------------------
    indices = np.where((State_Space_M == N_S_M).all(axis=1))[0]
    row_index_m = indices[0]
    col_index_m = 0
    return N_S_M, row_index_m, col_index_m, Soj_T




In [ ]:
delta = 0.25
State_Space_L={}
F1_N=[0,1]
F2_N=[0, 1]
F3_N=[0, 1]


State_Space_L = np.zeros((len(F1_N) * len(F2_N) * len(F3_N), 3))
k = 0
for i1 in range(len(F1_N)):
    for i2 in range(len(F2_N)):
        for i3 in range(len(F3_N)):
            state = np.zeros(3)
            state[0] = R_S_N[i1]
            state[1] = S_S_N[i2]
            state[2] = ST_S_N[i3]
            State_Space_L[k] = state
            k += 1

#---------Action Space------------------------
upper_bound = 1 + delta 
values = np.arange(0, upper_bound, delta)

values[values > 1] = 1            
            
Action_Space_L= values

In [ ]:
#-----------Edit-----------------------------
def Learning_Cost(IP, DL_Req, T_sog_me, CR, DR_R_R, CU, DR_R_U, CH, DR_R_H, CL, DR_R_S, a_h, a_m, a_l):
    
#============Task Processing===============================================
    out=[]
    V=IP[0]
    RSU_T=IP[1]
    UAV_T=IP[4]
    HAP_T=IP[7]
    LEO_T=IP[10]
    TS=IP[31]
    TSD=IP[32]
    psi_dmp=IP[33]
    Pcomp_m=IP[34]
    Ptp_v=IP[35]
    Pcomp_r=IP[36]
    Ptp_r=IP[37]
    Pcomp_u=IP[38]
    Ptp_u=IP[39]
    Pcomp_h=IP[40]
    Ptp_h=IP[41]
    Pcomp_s=IP[42]
    Ptp_s=IP[43]
    Cm=IP[44]

    #--------Local Device Computation----------------------------

    TPcomp_v=0
    EPcomp_v=0

    TPcomp_v=((TS[0][v_id]*psi_dmp)/(Cm[0][v_id]))
    EPcomp_v=(Pcomp_m[0][v_id]*TPcomp_v)

        
    #-----------Offloading Time and Energy------------------
    TPcomp_r=0
    EPcomp_r=0
    TPcomp_u=0
    EPcomp_u=0
    TPcomp_h=0
    EPcomp_h=0
    TPcomp_s=0
    EPcomp_s=0
    
    TPcomU_vr=0
    EPcomU_vr=0
    TPcomD_vr=0
    EPcomD_vr=0
    TPcomU_rv=0
    EPcomU_rv=0
    TPcomD_rv=0
    EPcomD_rv=0
    
    TPcomU_vu=0
    EPcomU_vu=0
    TPcomD_vu=0
    EPcomD_vu=0
    TPcomU_uv=0
    EPcomU_uv=0
    TPcomD_uv=0
    EPcomD_uv=0
    
    TPcomU_vh=0
    EPcomU_vh=0
    TPcomD_vh=0
    EPcomD_vh=0
    TPcomU_hv=0
    EPcomU_hv=0
    TPcomD_hv=0
    EPcomD_hv=0

    TPcomU_vs=0
    EPcomU_vs=0
    TPcomD_vs=0
    EPcomD_vs=0
    TPcomU_sv=0
    EPcomU_sv=0
    TPcomD_sv=0
    EPcomD_sv=0
    

    

    if(a_h==0):
        for r in range(RSU_T):
   
            if(np.nonzero(a_m)==r):
                TPcomp_r=((TS[0][v_id]*psi_dmp)/(CR[v_id][r]))
                EPcomp_r=(Pcomp_r[0][r]*TPcomp_r)
                TPcomU_vr=((TS[0][v_id])/DR_R_R)
                EPcomU_vr=TPcomU_vr*Ptp_v[0][v_id]
                EPcomU_rv=TPcomU_vr*Ptp_r[0][r]
                TPcomD_vr=((TSD[0][v_id])/DR_R_R)
                EPcomD_vr=TPcomD_vr*Ptp_v[0][v_id]
                EPcomD_rv=TPcomD_vr*Ptp_r[0][r]
                    
                    
                    
    #------------Total Task Processing Time and Energy--------------\n    
    TP_offl_vr=0
    EP_offl_vr=0
    TP_offl_rv=0
    EP_offl_rv=0
    TP_offl_T_r=0
    EP_offl_T_r=0   

    TP_loc=0
    EP_loc=0
 
    if(a_h==0):
        for r in range(RSU_T):
            if(np.nonzero(a_m)==r):
                TP_offl_vr=TPcomU_vr  + TPcomD_vr
                EP_offl_vr=EPcomU_vr  + EPcomD_vr
                TP_offl_rv=TPcomp_r
                EP_offl_rv=EPcomp_r

                TP_offl_T_r=TP_offl_vr  + TP_offl_rv
                EP_offl_T_r=0.5*EP_offl_vr  + 0.5*EP_offl_rv

                TP_loc=TPcomp_v
                EP_loc=0.5*EPcomp_v                    

                

    if(a_h==1):
        for u in range(UAV_T):
            if(np.nonzero(a_m)==u):
                TPcomp_u=((TS[0][v_id]*psi_dmp)/(CU[v_id][u]))
                EPcomp_u=(Pcomp_u[0][u]*TPcomp_u)

                TPcomU_vu=((TS[0][v_id])/DR_R_U)
                EPcomU_vu=TPcomU_vu*Ptp_v[0][v_id]
                EPcomU_uv=TPcomU_vu*Ptp_u[0][u]
                TPcomD_vu=((TSD[0][v_id])/DR_R_U)
                EPcomD_vu=TPcomD_vu*Ptp_v[0][v_id]
                EPcomD_uv=TPcomD_vu*Ptp_u[0][u]
                    
                 
                    
    #------------Total Task Processing Time and Energy (UAV)--------------\n    
    TP_offl_vu=0
    EP_offl_vu=0
    TP_offl_uv=0
    EP_offl_uv=0
    TP_offl_T_u=0
    EP_offl_T_u=0 


    for v in range(V):
        if(a_h==1):
            for u in range(UAV_T):
                if(np.nonzero(a_m)==u):
                    TP_offl_vu=TPcomU_vu  + TPcomD_vu
                    EP_offl_vu=EPcomU_vu  + EPcomD_vu
                    TP_offl_uv=TPcomp_u
                    EP_offl_uv=EPcomp_u
                    
                    TP_offl_T_u=TP_offl_vu  + TP_offl_uv
                    EP_offl_T_u=0.5*EP_offl_vu  + 0.5*EP_offl_uv

                    TP_loc=TPcomp_v
                    EP_loc=0.5*EPcomp_v                    

                    
                

    if(a_h==2):
        for h in range(HAP_T):
            if(np.nonzero(a_m)==h):
                TPcomp_h=((TS[0][v_id]*psi_dmp)/(CH[v_id][h]))
                EPcomp_h=(Pcomp_h[0][h]*TPcomp_h)

                TPcomU_vh=((TS[0][v_id])/DR_R_H)
                EPcomU_vh=TPcomU_vh*Ptp_v[0][v_id]
                EPcomU_hv=TPcomU_vh*Ptp_h[0][h]
                TPcomD_vh=((TSD[0][v_id])/DR_R_H)
                EPcomD_vh=TPcomD_vh*Ptp_v[0][v_id]
                EPcomD_hv=TPcomD_vh*Ptp_h[0][h]
                    
    #------------Total Task Processing Time and Energy (HAP)--------------\n    
    TP_offl_vh=0
    EP_offl_vh=0
    TP_offl_hv=0
    EP_offl_hv=0
    TP_offl_T_h=0
    EP_offl_T_h=0 


    for v in range(V):
        if(a_h==2):
            for h in range(HAP_T):
                if(np.nonzero(a_m)==h):
                    TP_offl_vh=TPcomU_vh  + TPcomD_vh
                    EP_offl_vh=EPcomU_vh  + EPcomD_vh
                    TP_offl_hv=TPcomp_h
                    EP_offl_hv=EPcomp_h
                    
                    TP_offl_T_h=TP_offl_vh  + TP_offl_hv
                    EP_offl_T_h=0.5*EP_offl_vh  + 0.5*EP_offl_hv

                    TP_loc=TPcomp_v
                    EP_loc=0.5*EPcomp_v  
                    
                    

                    
                    

    if(a_h==3):
        for s in range(LEO_T):
            if(np.nonzero(a_m)==s):
                TPcomp_s=((TS[0][v_id]*psi_dmp)/(CL[v_id][s]))
                EPcomp_s=(Pcomp_s[0][s]*TPcomp_s)

                TPcomU_vs=((TS[0][v_id])/DR_R_S)
                EPcomU_vs=TPcomU_vs*Ptp_v[0][v_id]
                EPcomU_sv=TPcomU_vs*Ptp_s[0][s]
                TPcomD_vs=((TSD[0][v_id])/DR_R_S)
                EPcomD_vs=TPcomD_vs*Ptp_v[0][v_id]
                EPcomD_sv=TPcomD_vs*Ptp_s[0][s]
                    
    
                    
    #------------Total Task Processing Time and Energy (HAP)--------------\n    
    TP_offl_vs=0
    EP_offl_vs=0
    TP_offl_sv=0
    EP_offl_sv=0
    TP_offl_T_s=0
    EP_offl_T_s=0  



    if(a_h==3):
        for s in range(LEO_T):
            if(np.nonzero(a_m)==s):
                TP_offl_vs=TPcomU_vs  + TPcomD_vs
                EP_offl_vs=EPcomU_vs  + EPcomD_vs
                TP_offl_sv=TPcomp_s
                EP_offl_sv=EPcomp_s

                TP_offl_T_s=TP_offl_vs  + TP_offl_sv
                EP_offl_T_s=0.5*EP_offl_vs  + 0.5*EP_offl_sv

                TP_loc=TPcomp_v
                EP_loc=0.5*EPcomp_v  
                    
                    
                    
                    
                    
    T_L=0
    T_L_V=0
    T_L_E=0
    T_E=0
    T_E_V=0
    T_E_E=0
    F1=0
    F2=0
    F3=0
    

    if(a_h==0):
        for r in range(RSU_T):
            if(np.nonzero(a_m)==r):
                T_L=max((a_l*TP_offl_T_r), ((1-a_l)*TP_loc))
                T_L_V= (1-a_l)*TP_loc
                T_L_E= (a_l*TP_offl_T_r)
                T_E=((a_l*EP_offl_T_r)+ ((1-a_l)*EP_loc))
                T_E_V= (1-a_l)*EP_loc
                T_E_E= (a_l*EP_offl_T_r)
                F1= T_L_V - T_sog_me
                F2= T_L - DL_Req
                F3= T_L_E - EP_loc
                
    if(a_h==1):
        for u in range(UAV_T):
            if(np.nonzero(a_m)==u):
                T_L=max((a_l*TP_offl_T_u), ((1-a_l)*TP_loc))
                T_E=((a_l*EP_offl_T_u)+ ((1-a_l)*EP_loc))  
                T_L_V= (1-a_l)*TP_loc
                T_L_E= (a_l*TP_offl_T_u)
                T_E_V= (1-a_l)*EP_loc
                T_E_E= (a_l*EP_offl_T_u)
                F1= T_L_V - T_sog_me
                F2= T_L - DL_Req
                F3= T_L_E - EP_loc        

    if(a_h==2):
        for h in range(HAP_T):
            if(np.nonzero(a_m)==h):
                T_L=max((a_l*TP_offl_T_h), ((1-a_l)*TP_loc))
                T_E=((a_l*EP_offl_T_h)+ ((1-a_l)*EP_loc))  
                T_L_V= (1-a_l)*TP_loc
                T_L_E= (a_l*TP_offl_T_h)
                T_E_V= (1-a_l)*EP_loc
                T_E_E= (a_l*EP_offl_T_h)
                F1= T_L_V - T_sog_me
                F2= T_L - DL_Req
                F3= T_L_E - EP_loc                    

    if(a_h==3):
        for s in range(LEO_T):
            if(np.nonzero(a_m)==s):
                T_L=max((a_l*TP_offl_T_s), ((1-a_l)*TP_loc))
                T_E=((a_l*EP_offl_T_s)+ ((1-a_l)*EP_loc))  
                T_L_V= (1-a_l)*TP_loc
                T_L_E= (a_l*TP_offl_T_s)
                T_E_V= (1-a_l)*EP_loc
                T_E_E= (a_l*EP_offl_T_s)
                F1= T_L_V - T_sog_me
                F2= T_L - DL_Req
                F3= T_L_E - EP_loc
                    
                    
    out.append(T_L)
    out.append(T_E)
    out.append(T_L_V)
    out.append(T_L_E)
    out.append(T_E_V)
    out.append(T_E_E)
    out.append(F1)
    out.append(F2)
    out.append(F3)
          
    return out



In [ ]:
num_episodes = 50
optimizer = 'adam'
state_size_h = len(State_Space_H)
action_size_h = len(Action_Space_H)
gamma_h = 0.7
epsilon_h = 0.1
agent_h = Agent(optimizer, state_size_h, action_size_h, gamma_h, epsilon_h)
agent_h1 = Agent(optimizer, state_size_h, action_size_h, gamma_h, epsilon_h)
agent_h2 = Agent(optimizer, state_size_h, action_size_h, gamma_h, epsilon_h)

state_size_m = len(State_Space_M)
action_size_m = RSU_T + UAV_T + HAP_T + LEO_T
gamma_m = 0.05
epsilon_m = 0.7
agent_m = Agent(optimizer, state_size_m, action_size_m, gamma_m, epsilon_m)
agent_m1 = Agent(optimizer, state_size_m, action_size_m, gamma_m, epsilon_m)
agent_m2 = Agent(optimizer, state_size_m, action_size_m, gamma_m, epsilon_m)

state_size_l = len(State_Space_L)
action_size_l = len(Action_Space_L)
gamma_l = 0.05
epsilon_l = 0.7
agent_l = Agent(optimizer, state_size_l, action_size_l, gamma_l, epsilon_l)
agent_l1 = Agent(optimizer, state_size_l, action_size_l, gamma_l, epsilon_l)
agent_l2 = Agent(optimizer, state_size_l, action_size_l, gamma_l, epsilon_l)



discount_factor = 0.98 #discount factor for future rewards
learning_rate = 0.05 #the rate at which the AI agent should learn
timesteps_per_episode = 1000
Max_U_RSU=8
Max_U_UAV=8
Max_U_HAP=20
Max_U_LEO=40
batch_size=2





In [ ]:
#-------LS DQN (0.5,0.5)-------------------------
environment_rows_h = len(State_Space_H)
environment_columns_h = len(Action_Space_H)
q_values_h = np.random.rand(environment_rows_h, environment_columns_h)
q_values_h = (q_values_h * 2000) - 1000
a_h=1

environment_rows_m = len(State_Space_M)
environment_columns_m = RSU_T + UAV_T + HAP_T + LEO_T
q_values_m = np.random.rand(environment_rows_m, environment_columns_m)
q_values_m = (q_values_m * 2000) - 1000

environment_rows_l = len(State_Space_L)
environment_columns_l = len(Action_Space_L)
q_values_l = np.random.rand(environment_rows_l, environment_columns_l)
q_values_l = (q_values_l * 2000) - 1000

reward_l=0
reward_h=0
reward_m=0

name_sv_ht='HT' 
name_sv_hq='HQ' 
name_sv_mt='MT' 
name_sv_mq='MQ' 
name_sv_lt='LT' 
name_sv_lq='LQ'

for episode in range(num_episodes):
    for e in range(0, 1):
        rewards_all_episodes_h =[]
        state_index_h= get_starting_location(environment_rows_h) 
        start_state_h=State_Space_H[state_index_h]

        rewards_all_episodes_m =[]
        state_index_m= get_starting_location(environment_rows_m) 
        start_state_m=State_Space_M[state_index_m]

        rewards_all_episodes_l =[]
        state_index_l= get_starting_location(environment_rows_l) 
        start_state_l=State_Space_L[state_index_l]

        check=0
        terminated = False
        v_id=0
        V_spd=VN_spd[v_id]
        Req_Ser=  random.choice([1,2,3,4,5,6])
        state_h=start_state_h
        for timestep in range(timesteps_per_episode):
            action_index_h =agent_h.act(state_h,q_values_h)
            #print('action_index_h',action_index_h)
            old_state_index_h = state_index_h
            new_state_h, new_state_index_h = Next_State_H ( action_index_h, Req_Ser, RSU_T, NO_VUs_RSU, Max_U_RSU, UAV_T, NO_VUs_UAV, Max_U_UAV, HAP_T, NO_VUs_HAP, Max_U_HAP, LEO_T, NO_VUs_LEO, Max_U_LEO)
            #print('new_state_h',new_state_h)
            M_A=M_Action_Space( action_index_h, v_id, VN_RSU_asign, VN_UAV_asign, VN_HAP_asign, VN_LEO_asign)
        
            nonzero_indices_M_A = np.nonzero(M_A)[1]
            if(action_index_h==0):
                nonzero_indices_M_A_l=nonzero_indices_M_A
                
            elif(action_index_h==1):
                nonzero_indices_M_A_l= RSU_T + nonzero_indices_M_A
            elif(action_index_h==2):
                nonzero_indices_M_A_l=RSU_T + UAV_T + nonzero_indices_M_A
                
            elif(action_index_h==3):
                nonzero_indices_M_A_l=RSU_T + UAV_T + HAP_T + nonzero_indices_M_A
            else:
                print('Error')
                

            action_index_m = get_next_action_M(state_index_m, nonzero_indices_M_A_l, epsilon_m, M_A, q_values_m) 

            nonzero_indices_m = np.nonzero(action_index_m)

            if(action_index_h==0):
                nonzero_indices_m_l=nonzero_indices_m[0][0]
                
            elif(action_index_h==1):
                nonzero_indices_m_l= RSU_T + nonzero_indices_m[0][0]
            elif(action_index_h==2):
                nonzero_indices_m_l=RSU_T + UAV_T + nonzero_indices_m[0][0]
                
            elif(action_index_h==3):
                nonzero_indices_m_l=RSU_T + UAV_T + HAP_T + nonzero_indices_m[0][0]
            else:
                print('Error')


            
            old_state_index_m = state_index_m

            new_state_m, new_state_index_m, col_index_m, T_soj_me =Next_State_M( action_index_h, action_index_m, V_spd, Req_Ser, NO_VUs_RSU, Max_U_RSU, service_allocation_RSU, 
                                                          VN_RSU_Soj[v_id], RSU_r, NO_VUs_UAV, Max_U_UAV, service_allocation_UAV, VN_UAV_Soj[v_id], 
                                                          UAV_r, NO_VUs_HAP, Max_U_HAP, service_allocation_HAP, VN_HAP_Soj[v_id], HAP_r, 
                                                          NO_VUs_LEO, Max_U_LEO, service_allocation_LEO, VN_LEO_Soj[v_id], LEO_r)

            #------DNN L--------
            action_index_l_id = get_next_action(state_index_l, epsilon_l, Action_Space_L, q_values_l) 
            action_l=Action_Space_L[int(action_index_l_id)]
            
            old_state_index_l = state_index_l
            out_l=Learning_Cost(VN_Main_IP, DL_Req, T_soj_me, RSU_C_RA, Rate_VR_R, UAV_C_RA, Rate_VU_R, HAP_C_RA, Rate_VH_R, 
                                LEO_C_RA, Rate_VL_R, action_index_h, action_index_m, action_l)

            Tot_LC=out_l[0]
            Tot_EC=out_l[1]
            Toc_LC_V=out_l[2]
            Toc_LC_E=out_l[3]
            Toc_EC_V=out_l[4]
            Toc_EC_E=out_l[5]
            F1_V=out_l[6]
            F2_V=out_l[7]
            F3_V=out_l[8]
            
            new_state_l=[0,0,0]
            if(F1_V > 0):
                new_state_l[0]=1
            if(F2_V > 0):
                new_state_l[1]=1 
            if(F3_V > 0):
                new_state_l[0]=1

            indices = np.argwhere((State_Space_L == new_state_l).all(axis=1))
            new_state_index_l=indices[0, 0]
            
            new_state_l_array = np.array(new_state_l)

            reward_l= 0.5*(Tot_LC) +  0.5*(Tot_EC)  
            reward_m=reward_l
            reward_h=reward_l

         
            agent_l.store(State_Space_L[old_state_index_l], action_index_l_id ,reward_l, new_state_l_array)            
            agent_m.store(State_Space_M[old_state_index_m], nonzero_indices_m_l ,reward_m, new_state_m)
            agent_h.store(State_Space_H[old_state_index_h], action_index_h ,reward_h, new_state_h)

            state_index_h = new_state_index_h
            state_index_m = new_state_index_m
            state_index_l = new_state_index_l


            if timestep == timesteps_per_episode - 1:
                agent_h.alighn_target_model() 
                if len(agent_h.experience_replay) > batch_size:
                    agent_h.retrain(batch_size)  
                if e % 10 == 0:
                    agent_h.save(name_sv_hq + '.weights.h5', name_sv_ht + '.weights.h5')
                    agent_m.save(name_sv_mq + '.weights.h5', name_sv_mt + '.weights.h5')
                    agent_l.save(name_sv_lq + '.weights.h5', name_sv_lt + '.weights.h5')


print('-------------------------------END--------------------------------------')
          

In [ ]:
#-------LS DQN (0.75,0.25)-------------------------
environment_rows_h = len(State_Space_H)
environment_columns_h = len(Action_Space_H)
q_values_h = np.random.rand(environment_rows_h, environment_columns_h)
q_values_h = (q_values_h * 2000) - 1000
a_h=1

environment_rows_m = len(State_Space_M)
environment_columns_m = RSU_T + UAV_T + HAP_T + LEO_T
q_values_m = np.random.rand(environment_rows_m, environment_columns_m)
q_values_m = (q_values_m * 2000) - 1000

environment_rows_l = len(State_Space_L)
environment_columns_l = len(Action_Space_L)
q_values_l = np.random.rand(environment_rows_l, environment_columns_l)
q_values_l = (q_values_l * 2000) - 1000

reward_l=0
reward_h=0
reward_m=0

name_sv_ht1='HT1' 
name_sv_hq1='HQ1' 
name_sv_mt1='MT1' 
name_sv_mq1='MQ1' 
name_sv_lt1='LT1' 
name_sv_lq1='LQ1'

for episode in range(num_episodes):
    for e in range(0, 1):
        rewards_all_episodes_h =[]
        state_index_h= get_starting_location(environment_rows_h) 
        start_state_h=State_Space_H[state_index_h]

        rewards_all_episodes_m =[]
        state_index_m= get_starting_location(environment_rows_m) 
        start_state_m=State_Space_M[state_index_m]

        rewards_all_episodes_l =[]
        state_index_l= get_starting_location(environment_rows_l) 
        start_state_l=State_Space_L[state_index_l]

        check=0
        terminated = False
        v_id=0
        V_spd=VN_spd[v_id]
        Req_Ser=  random.choice([1,2,3,4,5,6])
        state_h=start_state_h
        for timestep in range(timesteps_per_episode):
            action_index_h =agent_h1.act(state_h,q_values_h)
            #print('action_index_h',action_index_h)
            old_state_index_h = state_index_h
            new_state_h, new_state_index_h = Next_State_H ( action_index_h, Req_Ser, RSU_T, NO_VUs_RSU, Max_U_RSU, UAV_T, NO_VUs_UAV, Max_U_UAV, HAP_T, NO_VUs_HAP, Max_U_HAP, LEO_T, NO_VUs_LEO, Max_U_LEO)
            #print('new_state_h',new_state_h)
            M_A=M_Action_Space( action_index_h, v_id, VN_RSU_asign, VN_UAV_asign, VN_HAP_asign, VN_LEO_asign)
        
            nonzero_indices_M_A = np.nonzero(M_A)[1]
            if(action_index_h==0):
                nonzero_indices_M_A_l=nonzero_indices_M_A
                
            elif(action_index_h==1):
                nonzero_indices_M_A_l= RSU_T + nonzero_indices_M_A
            elif(action_index_h==2):
                nonzero_indices_M_A_l=RSU_T + UAV_T + nonzero_indices_M_A
                
            elif(action_index_h==3):
                nonzero_indices_M_A_l=RSU_T + UAV_T + HAP_T + nonzero_indices_M_A
            else:
                print('Error')
                

            action_index_m = get_next_action_M(state_index_m, nonzero_indices_M_A_l, epsilon_m, M_A, q_values_m) #-----check-----
            #print('action_index_m',action_index_m)

            nonzero_indices_m = np.nonzero(action_index_m)

            if(action_index_h==0):
                nonzero_indices_m_l=nonzero_indices_m[0][0]
                
            elif(action_index_h==1):
                nonzero_indices_m_l= RSU_T + nonzero_indices_m[0][0]
            elif(action_index_h==2):
                nonzero_indices_m_l=RSU_T + UAV_T + nonzero_indices_m[0][0]
                
            elif(action_index_h==3):
                nonzero_indices_m_l=RSU_T + UAV_T + HAP_T + nonzero_indices_m[0][0]
            else:
                print('Error')


            
            old_state_index_m = state_index_m

            new_state_m, new_state_index_m, col_index_m, T_soj_me =Next_State_M( action_index_h, action_index_m, V_spd, Req_Ser, NO_VUs_RSU, Max_U_RSU, service_allocation_RSU, 
                                                          VN_RSU_Soj[v_id], RSU_r, NO_VUs_UAV, Max_U_UAV, service_allocation_UAV, VN_UAV_Soj[v_id], 
                                                          UAV_r, NO_VUs_HAP, Max_U_HAP, service_allocation_HAP, VN_HAP_Soj[v_id], HAP_r, 
                                                          NO_VUs_LEO, Max_U_LEO, service_allocation_LEO, VN_LEO_Soj[v_id], LEO_r)

            #------DNN L--------
            action_index_l_id = get_next_action(state_index_l, epsilon_l, Action_Space_L, q_values_l) #-----check-----
            #print('action_index_l',action_index_l_id)

            #index = np.where(Action_Space_L == action_index_l)[0]
            action_l=Action_Space_L[int(action_index_l_id)]
            
            old_state_index_l = state_index_l
            #print(action_index_l_id, action_l)
            out_l=Learning_Cost(VN_Main_IP, DL_Req, T_soj_me, RSU_C_RA, Rate_VR_R, UAV_C_RA, Rate_VU_R, HAP_C_RA, Rate_VH_R, 
                                LEO_C_RA, Rate_VL_R, action_index_h, action_index_m, action_l)

            Tot_LC=out_l[0]
            Tot_EC=out_l[1]
            Toc_LC_V=out_l[2]
            Toc_LC_E=out_l[3]
            Toc_EC_V=out_l[4]
            Toc_EC_E=out_l[5]
            F1_V=out_l[6]
            F2_V=out_l[7]
            F3_V=out_l[8]
            
            new_state_l=[0,0,0]
            if(F1_V > 0):
                new_state_l[0]=1
            if(F2_V > 0):
                new_state_l[1]=1 
            if(F3_V > 0):
                new_state_l[0]=1

            indices = np.argwhere((State_Space_L == new_state_l).all(axis=1))
            new_state_index_l=indices[0, 0]
            #print('new_state_l',new_state_l,new_state_index_l)
            
            new_state_l_array = np.array(new_state_l)

            reward_l= 0.75*(Tot_LC) +  0.25*(Tot_EC) #---check------   
            reward_m=reward_l
            reward_h=reward_l

         
            agent_l1.store(State_Space_L[old_state_index_l], action_index_l_id ,reward_l, new_state_l_array)            
            agent_m1.store(State_Space_M[old_state_index_m], nonzero_indices_m_l ,reward_m, new_state_m)
            agent_h1.store(State_Space_H[old_state_index_h], action_index_h ,reward_h, new_state_h)

            state_index_h = new_state_index_h
            state_index_m = new_state_index_m
            state_index_l = new_state_index_l

            if timestep == timesteps_per_episode - 1:
                agent_h1.alighn_target_model() 
                if len(agent_h1.experience_replay) > batch_size:
                    agent_h1.retrain(batch_size)  
                if e % 10 == 0:
                    agent_h1.save(name_sv_hq1 + '.weights.h5', name_sv_ht1 + '.weights.h5')
                    agent_m1.save(name_sv_mq1 + '.weights.h5', name_sv_mt1 + '.weights.h5')
                    agent_l1.save(name_sv_lq1 + '.weights.h5', name_sv_lt1 + '.weights.h5')


print('-------------------------------END--------------------------------------')
          

In [ ]:
#-------LS DQN (0.75,0.25)-------------------------
environment_rows_h = len(State_Space_H)
environment_columns_h = len(Action_Space_H)
q_values_h = np.random.rand(environment_rows_h, environment_columns_h)
q_values_h = (q_values_h * 2000) - 1000
a_h=1

environment_rows_m = len(State_Space_M)
environment_columns_m = RSU_T + UAV_T + HAP_T + LEO_T
q_values_m = np.random.rand(environment_rows_m, environment_columns_m)
q_values_m = (q_values_m * 2000) - 1000

environment_rows_l = len(State_Space_L)
environment_columns_l = len(Action_Space_L)
q_values_l = np.random.rand(environment_rows_l, environment_columns_l)
q_values_l = (q_values_l * 2000) - 1000

reward_l=0
reward_h=0
reward_m=0

name_sv_ht2='HT2' 
name_sv_hq2='HQ2' 
name_sv_mt2='MT2' 
name_sv_mq2='MQ2' 
name_sv_lt2='LT2' 
name_sv_lq2='LQ2'

for episode in range(num_episodes):
    for e in range(0, 1):
        rewards_all_episodes_h =[]
        state_index_h= get_starting_location(environment_rows_h) 
        start_state_h=State_Space_H[state_index_h]

        rewards_all_episodes_m =[]
        state_index_m= get_starting_location(environment_rows_m) 
        start_state_m=State_Space_M[state_index_m]

        rewards_all_episodes_l =[]
        state_index_l= get_starting_location(environment_rows_l) 
        start_state_l=State_Space_L[state_index_l]

        check=0
        terminated = False
        v_id=0
        V_spd=VN_spd[v_id]
        Req_Ser=  random.choice([1,2,3,4,5,6])
        state_h=start_state_h
        for timestep in range(timesteps_per_episode):
            action_index_h =agent_h2.act(state_h,q_values_h)
            #print('action_index_h',action_index_h)
            old_state_index_h = state_index_h
            new_state_h, new_state_index_h = Next_State_H ( action_index_h, Req_Ser, RSU_T, NO_VUs_RSU, Max_U_RSU, UAV_T, NO_VUs_UAV, Max_U_UAV, HAP_T, NO_VUs_HAP, Max_U_HAP, LEO_T, NO_VUs_LEO, Max_U_LEO)
            #print('new_state_h',new_state_h)
            M_A=M_Action_Space( action_index_h, v_id, VN_RSU_asign, VN_UAV_asign, VN_HAP_asign, VN_LEO_asign)
        
            nonzero_indices_M_A = np.nonzero(M_A)[1]
            if(action_index_h==0):
                nonzero_indices_M_A_l=nonzero_indices_M_A
                
            elif(action_index_h==1):
                nonzero_indices_M_A_l= RSU_T + nonzero_indices_M_A
            elif(action_index_h==2):
                nonzero_indices_M_A_l=RSU_T + UAV_T + nonzero_indices_M_A
                
            elif(action_index_h==3):
                nonzero_indices_M_A_l=RSU_T + UAV_T + HAP_T + nonzero_indices_M_A
            else:
                print('Error')
                

            action_index_m = get_next_action_M(state_index_m, nonzero_indices_M_A_l, epsilon_m, M_A, q_values_m) #-----check-----
            #print('action_index_m',action_index_m)

            nonzero_indices_m = np.nonzero(action_index_m)

            if(action_index_h==0):
                nonzero_indices_m_l=nonzero_indices_m[0][0]
                
            elif(action_index_h==1):
                nonzero_indices_m_l= RSU_T + nonzero_indices_m[0][0]
            elif(action_index_h==2):
                nonzero_indices_m_l=RSU_T + UAV_T + nonzero_indices_m[0][0]
                
            elif(action_index_h==3):
                nonzero_indices_m_l=RSU_T + UAV_T + HAP_T + nonzero_indices_m[0][0]
            else:
                print('Error')


            
            old_state_index_m = state_index_m

            new_state_m, new_state_index_m, col_index_m, T_soj_me =Next_State_M( action_index_h, action_index_m, V_spd, Req_Ser, NO_VUs_RSU, Max_U_RSU, service_allocation_RSU, 
                                                          VN_RSU_Soj[v_id], RSU_r, NO_VUs_UAV, Max_U_UAV, service_allocation_UAV, VN_UAV_Soj[v_id], 
                                                          UAV_r, NO_VUs_HAP, Max_U_HAP, service_allocation_HAP, VN_HAP_Soj[v_id], HAP_r, 
                                                          NO_VUs_LEO, Max_U_LEO, service_allocation_LEO, VN_LEO_Soj[v_id], LEO_r)

            #------DNN L--------
            action_index_l_id = get_next_action(state_index_l, epsilon_l, Action_Space_L, q_values_l) #-----check-----
            #print('action_index_l',action_index_l_id)

            #index = np.where(Action_Space_L == action_index_l)[0]
            action_l=Action_Space_L[int(action_index_l_id)]
            
            old_state_index_l = state_index_l
            #print(action_index_l_id, action_l)
            out_l=Learning_Cost(VN_Main_IP, DL_Req, T_soj_me, RSU_C_RA, Rate_VR_R, UAV_C_RA, Rate_VU_R, HAP_C_RA, Rate_VH_R, 
                                LEO_C_RA, Rate_VL_R, action_index_h, action_index_m, action_l)

            Tot_LC=out_l[0]
            Tot_EC=out_l[1]
            Toc_LC_V=out_l[2]
            Toc_LC_E=out_l[3]
            Toc_EC_V=out_l[4]
            Toc_EC_E=out_l[5]
            F1_V=out_l[6]
            F2_V=out_l[7]
            F3_V=out_l[8]
            
            new_state_l=[0,0,0]
            if(F1_V > 0):
                new_state_l[0]=1
            if(F2_V > 0):
                new_state_l[1]=1 
            if(F3_V > 0):
                new_state_l[0]=1

            indices = np.argwhere((State_Space_L == new_state_l).all(axis=1))
            new_state_index_l=indices[0, 0]
            #print('new_state_l',new_state_l,new_state_index_l)
            
            new_state_l_array = np.array(new_state_l)

            reward_l= 0.75*(Tot_LC) +  0.25*(Tot_EC) #---check------   
            reward_m=reward_l
            reward_h=reward_l

         
            agent_l2.store(State_Space_L[old_state_index_l], action_index_l_id ,reward_l, new_state_l_array)            
            agent_m2.store(State_Space_M[old_state_index_m], nonzero_indices_m_l ,reward_m, new_state_m)
            agent_h2.store(State_Space_H[old_state_index_h], action_index_h ,reward_h, new_state_h)

            state_index_h = new_state_index_h
            state_index_m = new_state_index_m
            state_index_l = new_state_index_l

            if timestep == timesteps_per_episode - 1:
                agent_h2.alighn_target_model() 
                if len(agent_h2.experience_replay) > batch_size:
                    agent_h1.retrain(batch_size)  
                if e % 10 == 0:
                    agent_h2.save(name_sv_hq2 + '.weights.h5', name_sv_ht2 + '.weights.h5')
                    agent_m2.save(name_sv_mq2 + '.weights.h5', name_sv_mt2 + '.weights.h5')
                    agent_l2.save(name_sv_lq2 + '.weights.h5', name_sv_lt2 + '.weights.h5')


print('-------------------------------END--------------------------------------')
          

In [ ]:
agent_h.load(name_sv_hq + '.weights.h5', name_sv_ht + '.weights.h5')
agent_m.load(name_sv_mq + '.weights.h5', name_sv_mt + '.weights.h5')
agent_l.load(name_sv_lq + '.weights.h5', name_sv_lt + '.weights.h5')

agent_h1.load(name_sv_hq + '.weights.h5', name_sv_ht + '.weights.h5')
agent_m1.load(name_sv_mq + '.weights.h5', name_sv_mt + '.weights.h5')
agent_l1.load(name_sv_lq + '.weights.h5', name_sv_lt + '.weights.h5')

agent_h2.load(name_sv_hq + '.weights.h5', name_sv_ht + '.weights.h5')
agent_m2.load(name_sv_mq + '.weights.h5', name_sv_mt + '.weights.h5')
agent_l2.load(name_sv_lq + '.weights.h5', name_sv_lt + '.weights.h5')

In [ ]:
RP=10
Tot_V=[20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
       100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
       120, 120, 120, 120, 120, 120, 120, 120, 120, 120,
       140, 140, 140, 140, 140, 140, 140, 140, 140, 140,
       160, 160, 160, 160, 160, 160, 160, 160, 160, 160,
       180, 180, 180, 180, 180, 180, 180, 180, 180, 180,
       200, 200, 200, 200, 200, 200, 200, 200, 200, 200]

timesteps_per_episode=10
epsilon=1
Tot_T_L_RA=np.zeros(len(Tot_V))
Tot_T_E_RA=np.zeros(len(Tot_V))
Tot_T_L_V_RA=np.zeros(len(Tot_V))
Tot_T_L_E_RA=np.zeros(len(Tot_V))
Tot_T_E_V_RA=np.zeros(len(Tot_V))
Tot_T_E_E_RA=np.zeros(len(Tot_V))
Tot_TC_RA=np.zeros(len(Tot_V))
No_of_Ser_L_Fail_RA=np.zeros(len(Tot_V))
No_of_Soj_L_Fail_RA=np.zeros(len(Tot_V))
No_of_Ser_H_Req_RA=np.zeros(len(Tot_V))

Tot_T_L_SD_RA=np.zeros(len(Tot_V))
Tot_T_E_SD_RA=np.zeros(len(Tot_V))
Tot_T_L_V_SD_RA=np.zeros(len(Tot_V))
Tot_T_L_E_SD_RA=np.zeros(len(Tot_V))
Tot_T_E_V_SD_RA=np.zeros(len(Tot_V))
Tot_T_E_E_SD_RA=np.zeros(len(Tot_V))
Tot_TC_SD_RA=np.zeros(len(Tot_V))

Tot_T_L_HRL=np.zeros(len(Tot_V))
Tot_T_E_HRL=np.zeros(len(Tot_V))
Tot_T_L_V_HRL=np.zeros(len(Tot_V))
Tot_T_L_E_HRL=np.zeros(len(Tot_V))
Tot_T_E_V_HRL=np.zeros(len(Tot_V))
Tot_T_E_E_HRL=np.zeros(len(Tot_V))
Tot_TC_HRL=np.zeros(len(Tot_V))
No_of_Ser_L_Fail_HRL=np.zeros(len(Tot_V))
No_of_Soj_L_Fail_HRL=np.zeros(len(Tot_V))
No_of_Ser_H_Req_HRL=np.zeros(len(Tot_V))

Tot_T_L_SD_HRL=np.zeros(len(Tot_V))
Tot_T_E_SD_HRL=np.zeros(len(Tot_V))
Tot_T_L_V_SD_HRL=np.zeros(len(Tot_V))
Tot_T_L_E_SD_HRL=np.zeros(len(Tot_V))
Tot_T_E_V_SD_HRL=np.zeros(len(Tot_V))
Tot_T_E_E_SD_HRL=np.zeros(len(Tot_V))
Tot_TC_SD_HRL=np.zeros(len(Tot_V))

Tot_T_L_HRL1=np.zeros(len(Tot_V))
Tot_T_E_HRL1=np.zeros(len(Tot_V))
Tot_T_L_V_HRL1=np.zeros(len(Tot_V))
Tot_T_L_E_HRL1=np.zeros(len(Tot_V))
Tot_T_E_V_HRL1=np.zeros(len(Tot_V))
Tot_T_E_E_HRL1=np.zeros(len(Tot_V))
No_of_Ser_L_Fail_HRL1=np.zeros(len(Tot_V))
No_of_Soj_L_Fail_HRL1=np.zeros(len(Tot_V))
No_of_Ser_H_Req_HRL1=np.zeros(len(Tot_V))
Tot_TC_HRL1=np.zeros(len(Tot_V))

Tot_T_L_SD_HRL1=np.zeros(len(Tot_V))
Tot_T_E_SD_HRL1=np.zeros(len(Tot_V))
Tot_T_L_V_SD_HRL1=np.zeros(len(Tot_V))
Tot_T_L_E_SD_HRL1=np.zeros(len(Tot_V))
Tot_T_E_V_SD_HRL1=np.zeros(len(Tot_V))
Tot_T_E_E_SD_HRL1=np.zeros(len(Tot_V))
Tot_TC_SD_HRL1=np.zeros(len(Tot_V))

Tot_T_L_HRL2=np.zeros(len(Tot_V))
Tot_T_E_HRL2=np.zeros(len(Tot_V))
Tot_T_L_V_HRL2=np.zeros(len(Tot_V))
Tot_T_L_E_HRL2=np.zeros(len(Tot_V))
Tot_T_E_V_HRL2=np.zeros(len(Tot_V))
Tot_T_E_E_HRL2=np.zeros(len(Tot_V))
No_of_Ser_L_Fail_HRL2=np.zeros(len(Tot_V))
No_of_Soj_L_Fail_HRL2=np.zeros(len(Tot_V))
No_of_Ser_H_Req_HRL2=np.zeros(len(Tot_V))
Tot_TC_HRL2=np.zeros(len(Tot_V))

Tot_T_L_SD_HRL2=np.zeros(len(Tot_V))
Tot_T_E_SD_HRL2=np.zeros(len(Tot_V))
Tot_T_L_V_SD_HRL2=np.zeros(len(Tot_V))
Tot_T_L_E_SD_HRL2=np.zeros(len(Tot_V))
Tot_T_E_V_SD_HRL2=np.zeros(len(Tot_V))
Tot_T_E_E_SD_HRL2=np.zeros(len(Tot_V))
Tot_TC_SD_HRL2=np.zeros(len(Tot_V))

Tot_T_L_LC=np.zeros(len(Tot_V))
Tot_T_E_LC=np.zeros(len(Tot_V))
Tot_T_L_V_LC=np.zeros(len(Tot_V))
Tot_T_L_E_LC=np.zeros(len(Tot_V))
Tot_T_E_V_LC=np.zeros(len(Tot_V))
Tot_T_E_E_LC=np.zeros(len(Tot_V))
Tot_TC_LC=np.zeros(len(Tot_V))
No_of_Ser_L_Fail_LC=np.zeros(len(Tot_V))
No_of_Soj_L_Fail_LC=np.zeros(len(Tot_V))
No_of_Ser_H_Req_LC=np.zeros(len(Tot_V))

Tot_T_L_SD_LC=np.zeros(len(Tot_V))
Tot_T_E_SD_LC=np.zeros(len(Tot_V))
Tot_T_L_V_SD_LC=np.zeros(len(Tot_V))
Tot_T_L_E_SD_LC=np.zeros(len(Tot_V))
Tot_T_E_V_SD_LC=np.zeros(len(Tot_V))
Tot_T_E_E_SD_LC=np.zeros(len(Tot_V))
Tot_TC_SD_LC=np.zeros(len(Tot_V))

Tot_T_L_DA=np.zeros(len(Tot_V))
Tot_T_E_DA=np.zeros(len(Tot_V))
Tot_T_L_V_DA=np.zeros(len(Tot_V))
Tot_T_L_E_DA=np.zeros(len(Tot_V))
Tot_T_E_V_DA=np.zeros(len(Tot_V))
Tot_T_E_E_DA=np.zeros(len(Tot_V))
Tot_TC_DA=np.zeros(len(Tot_V))
No_of_Ser_L_Fail_DA=np.zeros(len(Tot_V))
No_of_Soj_L_Fail_DA=np.zeros(len(Tot_V))
No_of_Ser_H_Req_DA=np.zeros(len(Tot_V))


Tot_T_L_SD_DA=np.zeros(len(Tot_V))
Tot_T_E_SD_DA=np.zeros(len(Tot_V))
Tot_T_L_V_SD_DA=np.zeros(len(Tot_V))
Tot_T_L_E_SD_DA=np.zeros(len(Tot_V))
Tot_T_E_V_SD_DA=np.zeros(len(Tot_V))
Tot_T_E_E_SD_DA=np.zeros(len(Tot_V))
Tot_TC_SD_DA=np.zeros(len(Tot_V))

b_0=1
theta=3
for V_it in range(len(Tot_V)):
    print('V_it', V_it)
    print('--------------------------------')
    V=Tot_V[V_it]
    No_of_Ser=6
    C_V=(8*10**9)*np.ones((1, V))
    Pcomp_m=(0.8)*np.ones((1, V))
    psi_dmp=1000
    TS=(5*8*10**6)*np.ones((1, V))
    TSD=(1*8*10**6)*np.ones((1, V))
    DL_Req=4
    P_tx_v=(1.6)*np.ones((1, V))
    P_rx_v=(1.4)*np.ones((1, V))
    P_comv=(1)*np.ones((1, V))
    
    V_Ser_Req=np.zeros(V)
    for v in range(V):
        V_Ser_Req[v]=random.choice([1, 2, 3, 4, 5, 6])
    
    
    VN_spd=np.zeros(V)
    min_spd = 8   #
    max_spd = 14  
    mean = 12     
    sigma = 2    
    
    a, b = (min_spd - mean) / sigma, (max_spd - mean) / sigma
    trunc_dist = truncnorm(a, b, loc=mean, scale=sigma)
    
    for v in range(V):
        VN_spd[v] = trunc_dist.rvs(size=1)[0]
    
    VN_Main_IP=[]
    VN_Main_IP.append(V)
    VN_Main_IP.append(RSU_T)
    VN_Main_IP.append(RSU_loc)
    VN_Main_IP.append(RSU_r)
    VN_Main_IP.append(UAV_T)
    VN_Main_IP.append(UAV_loc)
    VN_Main_IP.append(UAV_r)
    VN_Main_IP.append(HAP_T)
    VN_Main_IP.append(HAP_loc)
    VN_Main_IP.append(HAP_r)
    VN_Main_IP.append(LEO_T)
    VN_Main_IP.append(LEO_loc)
    VN_Main_IP.append(LEO_r)
    VN_Main_IP.append(B_RSU)
    VN_Main_IP.append(B_UAV)
    VN_Main_IP.append(B_HAP)
    VN_Main_IP.append(B_LEO)
    VN_Main_IP.append(C_RSU)
    VN_Main_IP.append(C_UAV)
    VN_Main_IP.append(C_HAP)
    VN_Main_IP.append(C_LEO)
    VN_Main_IP.append(P_tx_r)
    VN_Main_IP.append(P_tx_u)
    VN_Main_IP.append(P_tx_h)
    VN_Main_IP.append(P_tx_s)
    VN_Main_IP.append(b_0)
    VN_Main_IP.append(theta)
    
    
    
    
    VN_assign_OUT =VN_EN_Assign(VN_Main_IP, VN_spd)
    
    VN_loc=VN_assign_OUT[0]
    VN_RSU_asign=VN_assign_OUT[1]
    VN_RSU_Soj=VN_assign_OUT[2]
    VN_RSU_dist=VN_assign_OUT[3]
    VN_UAV_asign=VN_assign_OUT[4]
    VN_UAV_Soj=VN_assign_OUT[5]
    VN_UAV_dist=VN_assign_OUT[6]
    VN_HAP_asign=VN_assign_OUT[7]
    VN_HAP_Soj=VN_assign_OUT[8]
    VN_HAP_dist=VN_assign_OUT[9]
    VN_LEO_asign=VN_assign_OUT[10]
    VN_LEO_Soj=VN_assign_OUT[11]
    VN_LEO_dist=VN_assign_OUT[12]
    VN_RSU_dist_Sj=VN_assign_OUT[13]
    VN_UAV_dist_Sj=VN_assign_OUT[14]
    VN_HAP_dist_Sj=VN_assign_OUT[15]
    VN_LEO_dist_Sj=VN_assign_OUT[16]
    
    VN_Main_IP.append(VN_RSU_dist)
    VN_Main_IP.append(VN_UAV_dist)
    VN_Main_IP.append(VN_HAP_dist)
    VN_Main_IP.append(VN_LEO_dist)
    VN_Main_IP.append(TS)
    VN_Main_IP.append(TSD)
    VN_Main_IP.append(psi_dmp)
    VN_Main_IP.append(Pcomp_m)
    VN_Main_IP.append(P_tx_v)
    VN_Main_IP.append(Pcomp_r)
    VN_Main_IP.append(P_tx_r)
    VN_Main_IP.append(Pcomp_u)
    VN_Main_IP.append(P_tx_u)
    VN_Main_IP.append(Pcomp_h)
    VN_Main_IP.append(P_tx_h)
    VN_Main_IP.append(Pcomp_s)
    VN_Main_IP.append(P_tx_s)
    VN_Main_IP.append(C_V)
    VN_Main_IP.append(VN_RSU_Soj)
    VN_Main_IP.append(VN_UAV_Soj)
    VN_Main_IP.append(VN_HAP_Soj)
    VN_Main_IP.append(VN_LEO_Soj)
    VN_Main_IP.append(gamma_1)
    VN_Main_IP.append(gamma_2)
    VN_Main_IP.append(service_allocation_RSU)
    VN_Main_IP.append(service_allocation_UAV)
    VN_Main_IP.append(service_allocation_HAP)
    VN_Main_IP.append(service_allocation_LEO)
    VN_Main_IP.append(VN_loc)

    #--------Random Approach--------------------------
    #1) Layer Selection
    Layer_Selection_random=np.zeros(V)
    Node_selection_random=np.zeros(V)
    Offloading_random=np.zeros(V)
    
    Random_decisions=np.zeros((V,3))
    
        
    
    for v in range(V):
        Random_decisions[v][0] = random.choice([0, 1, 2, 3])
        if(Random_decisions[v][0] ==0):
            Node_seelction_choice=VN_RSU_asign[v]
            indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
            Random_decisions[v][1] = random.choice(indices_with_ones.tolist())
            Random_decisions[v][2]=random.random()
            
        if(Random_decisions[v][0] ==1):
            Node_seelction_choice=VN_UAV_asign[v]
            indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
            Random_decisions[v][1] = random.choice(indices_with_ones.tolist())
            Random_decisions[v][2]=random.random()
    
        if(Random_decisions[v][0] ==2):
            Node_seelction_choice=VN_HAP_asign[v]
            indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
            Random_decisions[v][1] = random.choice(indices_with_ones.tolist())
            Random_decisions[v][2]=random.random()
            
        if(Random_decisions[v][0] ==3):
            Node_seelction_choice=VN_LEO_asign[v]
            indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
            Random_decisions[v][1] = random.choice(indices_with_ones.tolist())
            Random_decisions[v][2]=random.random()
    
    NO_VUs_RSU=np.zeros(RSU_T)
    NO_VUs_UAV=np.zeros(UAV_T)
    NO_VUs_HAP=np.zeros(HAP_T)
    NO_VUs_LEO=np.zeros(LEO_T)
    for v in range(V):
        if(Random_decisions[v][0]==0):
            for r in range(RSU_T):
                if(Random_decisions[v][1]==r):
                    NO_VUs_RSU[r]=NO_VUs_RSU[r]+1
    
        if(Random_decisions[v][0]==1):
            for u in range(UAV_T):
                if(Random_decisions[v][1]==u):
                    NO_VUs_UAV[u]=NO_VUs_UAV[u]+1
    
        if(Random_decisions[v][0]==2):
            for h in range(HAP_T):
                if(Random_decisions[v][1]==h):
                    NO_VUs_HAP[h]=NO_VUs_HAP[h]+1
                    
        if(Random_decisions[v][0]==3):
            for s in range(LEO_T):
                if(Random_decisions[v][1]==s):
                    NO_VUs_LEO[s]=NO_VUs_LEO[s]+1
    
    #-----Resource Allocation------------
    RA_Random_OUT= Resource_Allocation( VN_Main_IP,  Random_decisions, NO_VUs_RSU, NO_VUs_UAV, NO_VUs_HAP, NO_VUs_LEO)
    
    RSU_B_RA=RA_Random_OUT[0]
    UAV_B_RA=RA_Random_OUT[1]
    HAP_B_RA=RA_Random_OUT[2]
    LEO_B_RA=RA_Random_OUT[3]
    RSU_C_RA=RA_Random_OUT[4]
    UAV_C_RA=RA_Random_OUT[5]
    HAP_C_RA=RA_Random_OUT[6]
    LEO_C_RA=RA_Random_OUT[7]
    
    Rate_VR_R, Rate_VU_R, Rate_VH_R, Rate_VL_R= Data_Rate(VN_Main_IP,Random_decisions, RSU_B_RA, UAV_B_RA, HAP_B_RA, LEO_B_RA)
        
    Cost=Task_Proc_Main(VN_Main_IP, RSU_C_RA, Rate_VR_R, UAV_C_RA, Rate_VU_R, HAP_C_RA, Rate_VH_R, LEO_C_RA, Rate_VL_R, 0.5, 0.5, Random_decisions)
    T_L_RA=Cost[0]
    T_E_RA=Cost[1]
    T_L_V_RA=Cost[2]
    T_L_E_RA=Cost[3]
    T_E_V_RA=Cost[4]
    T_E_E_RA=Cost[5]
    TC_RA=Cost[8]
    
    No_of_Ser_L_Fail_RA[V_it]=Cost[9]
    No_of_Soj_L_Fail_RA[V_it]=Cost[10]
    No_of_Ser_H_Req_RA[V_it]=Cost[11]


    
    Tot_TC_RA[V_it]=np.mean(TC_RA)
    Tot_T_L_RA[V_it]=np.mean(T_L_RA)
    Tot_T_E_RA[V_it]=np.mean(T_E_RA)
    Tot_T_L_V_RA[V_it]=np.mean(T_L_V_RA)
    Tot_T_L_E_RA[V_it]=np.mean(T_L_E_RA)
    Tot_T_E_V_RA[V_it]=np.mean(T_E_V_RA)
    Tot_T_E_E_RA[V_it]=np.mean(T_E_E_RA)
    
    Tot_TC_SD_RA[V_it]=np.std(TC_RA)
    Tot_T_L_SD_RA[V_it]=np.std(T_L_RA)
    Tot_T_E_SD_RA[V_it]=np.std(T_E_RA)
    Tot_T_L_V_SD_RA[V_it]=np.std(T_L_V_RA)
    Tot_T_L_E_SD_RA[V_it]=np.std(T_L_E_RA)
    Tot_T_E_V_SD_RA[V_it]=np.std(T_E_V_RA)
    Tot_T_E_E_SD_RA[V_it]=np.std(T_E_E_RA)
    
    
    #---------------HRL Solutions (0.5,0.5)----------------------------
    HRL_decisions=np.zeros((V,3))
    for v in range(V):
        state_index_h = get_starting_location(environment_rows_h) 
        state_h = State_Space_H[state_index_h]
        
        state_index_m = get_starting_location(environment_rows_m) 
        state_m = State_Space_M[state_index_m]
        
        state_index_l = get_starting_location(environment_rows_l) 
        state_l = State_Space_L[state_index_l]
        
        v_id =v
        V_spd = VN_spd[v]
        Req_Ser =V_Ser_Req[v]
    
        for timestep in range(timesteps_per_episode):
        
            action_index_h = agent_h.act(state_h, q_values_h)
            new_state_h, new_state_index_h = Next_State_H(action_index_h, Req_Ser, RSU_T, NO_VUs_RSU, Max_U_RSU, UAV_T, NO_VUs_UAV, Max_U_UAV, HAP_T, NO_VUs_HAP, Max_U_HAP, LEO_T, NO_VUs_LEO, Max_U_LEO)
            
            M_A = M_Action_Space(action_index_h, v_id, VN_RSU_asign, VN_UAV_asign, VN_HAP_asign, VN_LEO_asign)
            nonzero_indices_M_A = np.nonzero(M_A)[1]
            
            if action_index_h == 0:
                nonzero_indices_M_A_l = nonzero_indices_M_A
            elif action_index_h == 1:
                nonzero_indices_M_A_l = RSU_T + nonzero_indices_M_A
            elif action_index_h == 2:
                nonzero_indices_M_A_l = RSU_T + UAV_T + nonzero_indices_M_A
            elif action_index_h == 3:
                nonzero_indices_M_A_l = RSU_T + UAV_T + HAP_T + nonzero_indices_M_A
            
            action_index_m = get_next_action_M(state_index_m, nonzero_indices_M_A_l, epsilon_m, M_A, q_values_m)
            new_state_m, new_state_index_m, col_index_m, T_soj_me = Next_State_M(action_index_h, action_index_m, V_spd, Req_Ser, NO_VUs_RSU, Max_U_RSU, service_allocation_RSU, VN_RSU_Soj[v_id], RSU_r, NO_VUs_UAV, Max_U_UAV, service_allocation_UAV, VN_UAV_Soj[v_id], UAV_r, NO_VUs_HAP, Max_U_HAP, service_allocation_HAP, VN_HAP_Soj[v_id], HAP_r, NO_VUs_LEO, Max_U_LEO, service_allocation_LEO, VN_LEO_Soj[v_id], LEO_r)
        
            action_index_l_id = agent_l.act(state_l, q_values_l)
            
            action_l = Action_Space_L[int(action_index_l_id)]
            out_l = Learning_Cost(VN_Main_IP, DL_Req, T_soj_me, RSU_C_RA, Rate_VR_R, UAV_C_RA, Rate_VU_R, HAP_C_RA, Rate_VH_R, LEO_C_RA, Rate_VL_R, action_index_h, action_index_m, action_index_l_id)
            
            Tot_LC = out_l[0]
            Tot_EC = out_l[1]
            Toc_LC_V = out_l[2]
            Toc_LC_E = out_l[3]
            Toc_EC_V = out_l[4]
            Toc_EC_E = out_l[5]
            F1_V = out_l[6]
            F2_V = out_l[7]
            F3_V = out_l[8]
            
            new_state_l = [0, 0, 0]
            if F1_V > 0:
                new_state_l[0] = 1
            if F2_V > 0:
                new_state_l[1] = 1
            if F3_V > 0:
                new_state_l[0] = 1
            
            indices = np.argwhere((State_Space_L == new_state_l).all(axis=1))
            new_state_index_l = indices[0, 0]
            
            state_index_h = new_state_index_h
            state_index_m = new_state_index_m
            state_index_l = new_state_index_l
            
            state_h = State_Space_H[state_index_h]
            state_m = State_Space_M[state_index_m]
            state_l = State_Space_L[state_index_l]

            
    
        HRL_decisions[v][0] = action_index_h
        if(action_index_h ==0):
            Node_seelction_choice=action_index_m
            indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
            HRL_decisions[v][1] = random.choice(indices_with_ones.tolist())
            HRL_decisions[v][2]=action_l
            
        if(action_index_h ==1):
            Node_seelction_choice=action_index_m
            indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
            HRL_decisions[v][1] = random.choice(indices_with_ones.tolist())
            HRL_decisions[v][2]=action_l
    
        if(action_index_h ==2):
            Node_seelction_choice=action_index_m
            indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
            HRL_decisions[v][1] = random.choice(indices_with_ones.tolist())
            HRL_decisions[v][2]=action_l
            
        if(action_index_h ==3):
            Node_seelction_choice=action_index_m
            indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
            HRL_decisions[v][1] = random.choice(indices_with_ones.tolist())
            HRL_decisions[v][2]=action_l
    
    
    
    NO_VUs_RSU_HRL=np.zeros(RSU_T)
    NO_VUs_UAV_HRL=np.zeros(UAV_T)
    NO_VUs_HAP_HRL=np.zeros(HAP_T)
    NO_VUs_LEO_HRL=np.zeros(LEO_T)
    for v in range(V):
        if(HRL_decisions[v][0]==0):
            for r in range(RSU_T):
                if(HRL_decisions[v][1]==r):
                    NO_VUs_RSU_HRL[r]=NO_VUs_RSU_HRL[r]+1
    
        if(HRL_decisions[v][0]==1):
            for u in range(UAV_T):
                if(HRL_decisions[v][1]==u):
                    NO_VUs_UAV_HRL[u]=NO_VUs_UAV_HRL[u]+1
    
        if(HRL_decisions[v][0]==2):
            for h in range(HAP_T):
                if(HRL_decisions[v][1]==h):
                    NO_VUs_HAP_HRL[h]=NO_VUs_HAP_HRL[h]+1
                    
        if(HRL_decisions[v][0]==3):
            for s in range(LEO_T):
                if(HRL_decisions[v][1]==s):
                    NO_VUs_LEO_HRL[s]=NO_VUs_LEO_HRL[s]+1
    
    
    #-----Resource Allocation------------
    RA_HRL_OUT= Resource_Allocation( VN_Main_IP,  HRL_decisions, NO_VUs_RSU_HRL, NO_VUs_UAV_HRL, NO_VUs_HAP_HRL, NO_VUs_LEO_HRL)
    
    RSU_B_HRL=RA_HRL_OUT[0]
    UAV_B_HRL=RA_HRL_OUT[1]
    HAP_B_HRL=RA_HRL_OUT[2]
    LEO_B_HRL=RA_HRL_OUT[3]
    RSU_C_HRL=RA_HRL_OUT[4]
    UAV_C_HRL=RA_HRL_OUT[5]
    HAP_C_HRL=RA_HRL_OUT[6]
    LEO_C_HRL=RA_HRL_OUT[7]
    
    Rate_VR_HRL, Rate_VU_HRL, Rate_VH_HRL, Rate_VL_HRL= Data_Rate(VN_Main_IP,HRL_decisions, RSU_B_HRL, UAV_B_HRL, HAP_B_HRL, LEO_B_HRL)
        
    Cost_HRL=Task_Proc_Main(VN_Main_IP, RSU_C_HRL, Rate_VR_HRL, UAV_C_HRL, Rate_VU_HRL, HAP_C_HRL, Rate_VH_HRL, LEO_C_HRL, Rate_VL_HRL, 0.5, 0.5, HRL_decisions)
    T_L_HRL=Cost_HRL[0]
    T_E_HRL=Cost_HRL[1]
    T_L_V_HRL=Cost_HRL[2]
    T_L_E_HRL=Cost_HRL[3]
    T_E_V_HRL=Cost_HRL[4]
    T_E_E_HRL=Cost_HRL[5]
    TC_HRL=Cost_HRL[8]
    
    No_of_Ser_L_Fail_HRL[V_it]=Cost_HRL[9]
    No_of_Soj_L_Fail_HRL[V_it]=Cost_HRL[10]
    No_of_Ser_H_Req_HRL[V_it]=Cost_HRL[11]
    
    
    Tot_TC_HRL[V_it]=np.mean(TC_HRL)
    Tot_T_L_HRL[V_it]=np.mean(T_L_HRL)
    Tot_T_E_HRL[V_it]=np.mean(T_E_HRL)
    Tot_T_L_V_HRL[V_it]=np.mean(T_L_V_HRL)
    Tot_T_L_E_HRL[V_it]=np.mean(T_L_E_HRL)
    Tot_T_E_V_HRL[V_it]=np.mean(T_E_V_HRL)
    Tot_T_E_E_HRL[V_it]=np.mean(T_E_E_HRL)

    
    Tot_TC_SD_HRL[V_it]=np.std(TC_HRL)
    Tot_T_L_SD_HRL[V_it]=np.std(T_L_HRL)
    Tot_T_E_SD_HRL[V_it]=np.std(T_E_HRL)
    Tot_T_L_V_SD_HRL[V_it]=np.std(T_L_V_HRL)
    Tot_T_L_E_SD_HRL[V_it]=np.std(T_L_E_HRL)
    Tot_T_E_V_SD_HRL[V_it]=np.std(T_E_V_HRL)
    Tot_T_E_E_SD_HRL[V_it]=np.std(T_E_E_HRL)


    #---------------HRL Solutions (0.5,0.5)----------------------------
    HRL_decisions1=np.zeros((V,3))
    for v in range(V):
        state_index_h = get_starting_location(environment_rows_h) 
        state_h = State_Space_H[state_index_h]
        
        state_index_m = get_starting_location(environment_rows_m) 
        state_m = State_Space_M[state_index_m]
        
        state_index_l = get_starting_location(environment_rows_l) 
        state_l = State_Space_L[state_index_l]
        
        v_id =v
        V_spd = VN_spd[v]
        Req_Ser =V_Ser_Req[v]
    
        for timestep in range(timesteps_per_episode):
        
            action_index_h = agent_h1.act(state_h, q_values_h)
            new_state_h, new_state_index_h = Next_State_H(action_index_h, Req_Ser, RSU_T, NO_VUs_RSU, Max_U_RSU, UAV_T, NO_VUs_UAV, Max_U_UAV, HAP_T, NO_VUs_HAP, Max_U_HAP, LEO_T, NO_VUs_LEO, Max_U_LEO)
            
            M_A = M_Action_Space(action_index_h, v_id, VN_RSU_asign, VN_UAV_asign, VN_HAP_asign, VN_LEO_asign)
            nonzero_indices_M_A = np.nonzero(M_A)[1]
            
            if action_index_h == 0:
                nonzero_indices_M_A_l = nonzero_indices_M_A
            elif action_index_h == 1:
                nonzero_indices_M_A_l = RSU_T + nonzero_indices_M_A
            elif action_index_h == 2:
                nonzero_indices_M_A_l = RSU_T + UAV_T + nonzero_indices_M_A
            elif action_index_h == 3:
                nonzero_indices_M_A_l = RSU_T + UAV_T + HAP_T + nonzero_indices_M_A
            
            action_index_m = get_next_action_M(state_index_m, nonzero_indices_M_A_l, epsilon_m, M_A, q_values_m)
            new_state_m, new_state_index_m, col_index_m, T_soj_me = Next_State_M(action_index_h, action_index_m, V_spd, Req_Ser, NO_VUs_RSU, Max_U_RSU, service_allocation_RSU, VN_RSU_Soj[v_id], RSU_r, NO_VUs_UAV, Max_U_UAV, service_allocation_UAV, VN_UAV_Soj[v_id], UAV_r, NO_VUs_HAP, Max_U_HAP, service_allocation_HAP, VN_HAP_Soj[v_id], HAP_r, NO_VUs_LEO, Max_U_LEO, service_allocation_LEO, VN_LEO_Soj[v_id], LEO_r)
        
            action_index_l_id = agent_l1.act(state_l, q_values_l)
            
            action_l = Action_Space_L[int(action_index_l_id)]
            out_l = Learning_Cost(VN_Main_IP, DL_Req, T_soj_me, RSU_C_RA, Rate_VR_R, UAV_C_RA, Rate_VU_R, HAP_C_RA, Rate_VH_R, LEO_C_RA, Rate_VL_R, action_index_h, action_index_m, action_index_l_id)
            
            Tot_LC = out_l[0]
            Tot_EC = out_l[1]
            Toc_LC_V = out_l[2]
            Toc_LC_E = out_l[3]
            Toc_EC_V = out_l[4]
            Toc_EC_E = out_l[5]
            F1_V = out_l[6]
            F2_V = out_l[7]
            F3_V = out_l[8]
            
            new_state_l = [0, 0, 0]
            if F1_V > 0:
                new_state_l[0] = 1
            if F2_V > 0:
                new_state_l[1] = 1
            if F3_V > 0:
                new_state_l[0] = 1
            
            indices = np.argwhere((State_Space_L == new_state_l).all(axis=1))
            new_state_index_l = indices[0, 0]
            
            state_index_h = new_state_index_h
            state_index_m = new_state_index_m
            state_index_l = new_state_index_l
            
            state_h = State_Space_H[state_index_h]
            state_m = State_Space_M[state_index_m]
            state_l = State_Space_L[state_index_l]

            
    
        HRL_decisions1[v][0] = action_index_h
        if(action_index_h ==0):
            Node_seelction_choice=action_index_m
            indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
            HRL_decisions1[v][1] = random.choice(indices_with_ones.tolist())
            HRL_decisions1[v][2]=action_l
            
        if(action_index_h ==1):
            Node_seelction_choice=action_index_m
            indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
            HRL_decisions1[v][1] = random.choice(indices_with_ones.tolist())
            HRL_decisions1[v][2]=action_l
    
        if(action_index_h ==2):
            Node_seelction_choice=action_index_m
            indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
            HRL_decisions1[v][1] = random.choice(indices_with_ones.tolist())
            HRL_decisions1[v][2]=action_l
            
        if(action_index_h ==3):
            Node_seelction_choice=action_index_m
            indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
            HRL_decisions1[v][1] = random.choice(indices_with_ones.tolist())
            HRL_decisions1[v][2]=action_l
    
    
    
    NO_VUs_RSU_HRL1=np.zeros(RSU_T)
    NO_VUs_UAV_HRL1=np.zeros(UAV_T)
    NO_VUs_HAP_HRL1=np.zeros(HAP_T)
    NO_VUs_LEO_HRL1=np.zeros(LEO_T)
    for v in range(V):
        if(HRL_decisions1[v][0]==0):
            for r in range(RSU_T):
                if(HRL_decisions1[v][1]==r):
                    NO_VUs_RSU_HRL1[r]=NO_VUs_RSU_HRL1[r]+1
    
        if(HRL_decisions1[v][0]==1):
            for u in range(UAV_T):
                if(HRL_decisions1[v][1]==u):
                    NO_VUs_UAV_HRL1[u]=NO_VUs_UAV_HRL1[u]+1
    
        if(HRL_decisions1[v][0]==2):
            for h in range(HAP_T):
                if(HRL_decisions1[v][1]==h):
                    NO_VUs_HAP_HRL1[h]=NO_VUs_HAP_HRL1[h]+1
                    
        if(HRL_decisions1[v][0]==3):
            for s in range(LEO_T):
                if(HRL_decisions1[v][1]==s):
                    NO_VUs_LEO_HRL1[s]=NO_VUs_LEO_HRL1[s]+1
    
    
    #-----Resource Allocation------------
    RA_HRL1_OUT= Resource_Allocation( VN_Main_IP,  HRL_decisions1, NO_VUs_RSU_HRL1, NO_VUs_UAV_HRL1, NO_VUs_HAP_HRL1, NO_VUs_LEO_HRL1)
    
    RSU_B_HRL1=RA_HRL1_OUT[0]
    UAV_B_HRL1=RA_HRL1_OUT[1]
    HAP_B_HRL1=RA_HRL1_OUT[2]
    LEO_B_HRL1=RA_HRL1_OUT[3]
    RSU_C_HRL1=RA_HRL1_OUT[4]
    UAV_C_HRL1=RA_HRL1_OUT[5]
    HAP_C_HRL1=RA_HRL1_OUT[6]
    LEO_C_HRL1=RA_HRL1_OUT[7]
    
    Rate_VR_HRL1, Rate_VU_HRL1, Rate_VH_HRL1, Rate_VL_HRL1= Data_Rate(VN_Main_IP,HRL_decisions1, RSU_B_HRL1, UAV_B_HRL1, HAP_B_HRL1, LEO_B_HRL1)
        
    Cost_HRL1=Task_Proc_Main(VN_Main_IP, RSU_C_HRL1, Rate_VR_HRL1, UAV_C_HRL1, Rate_VU_HRL1, HAP_C_HRL1, Rate_VH_HRL1, LEO_C_HRL1, Rate_VL_HRL1, 0.75, 0.25, HRL_decisions1)
    T_L_HRL1=Cost_HRL1[0]
    T_E_HRL1=Cost_HRL1[1]
    T_L_V_HRL1=Cost_HRL1[2]
    T_L_E_HRL1=Cost_HRL1[3]
    T_E_V_HRL1=Cost_HRL1[4]
    T_E_E_HRL1=Cost_HRL1[5]
    TC_HRL1=Cost_HRL1[8]
    No_of_Ser_L_Fail_HRL1[V_it]=Cost_HRL1[9]
    No_of_Soj_L_Fail_HRL1[V_it]=Cost_HRL1[10]
    No_of_Ser_H_Req_HRL1[V_it]=Cost_HRL1[11]
    
    
    Tot_TC_HRL1[V_it]=np.mean(TC_HRL1)
    Tot_T_L_HRL1[V_it]=np.mean(T_L_HRL1)
    Tot_T_E_HRL1[V_it]=np.mean(T_E_HRL1)
    Tot_T_L_V_HRL1[V_it]=np.mean(T_L_V_HRL1)
    Tot_T_L_E_HRL1[V_it]=np.mean(T_L_E_HRL1)
    Tot_T_E_V_HRL1[V_it]=np.mean(T_E_V_HRL1)
    Tot_T_E_E_HRL1[V_it]=np.mean(T_E_E_HRL1)

    Tot_TC_SD_HRL[V_it]=np.std(TC_HRL)
    Tot_T_L_SD_HRL1[V_it]=np.std(T_L_HRL1)
    Tot_T_E_SD_HRL1[V_it]=np.std(T_E_HRL1)
    Tot_T_L_V_SD_HRL1[V_it]=np.std(T_L_V_HRL1)
    Tot_T_L_E_SD_HRL1[V_it]=np.std(T_L_E_HRL1)
    Tot_T_E_V_SD_HRL1[V_it]=np.std(T_E_V_HRL1)
    Tot_T_E_E_SD_HRL1[V_it]=np.std(T_E_E_HRL1)

    #---------------HRL Solutions (0.5,0.5)----------------------------
    HRL_decisions2=np.zeros((V,3))
    for v in range(V):
        state_index_h = get_starting_location(environment_rows_h) 
        state_h = State_Space_H[state_index_h]
        
        state_index_m = get_starting_location(environment_rows_m) 
        state_m = State_Space_M[state_index_m]
        
        state_index_l = get_starting_location(environment_rows_l) 
        state_l = State_Space_L[state_index_l]
        
        v_id =v
        V_spd = VN_spd[v]
        Req_Ser =V_Ser_Req[v]
    
        for timestep in range(timesteps_per_episode):
        
            action_index_h = agent_h2.act(state_h, q_values_h)
            new_state_h, new_state_index_h = Next_State_H(action_index_h, Req_Ser, RSU_T, NO_VUs_RSU, Max_U_RSU, UAV_T, NO_VUs_UAV, Max_U_UAV, HAP_T, NO_VUs_HAP, Max_U_HAP, LEO_T, NO_VUs_LEO, Max_U_LEO)
            
            M_A = M_Action_Space(action_index_h, v_id, VN_RSU_asign, VN_UAV_asign, VN_HAP_asign, VN_LEO_asign)
            nonzero_indices_M_A = np.nonzero(M_A)[1]
            
            if action_index_h == 0:
                nonzero_indices_M_A_l = nonzero_indices_M_A
            elif action_index_h == 1:
                nonzero_indices_M_A_l = RSU_T + nonzero_indices_M_A
            elif action_index_h == 2:
                nonzero_indices_M_A_l = RSU_T + UAV_T + nonzero_indices_M_A
            elif action_index_h == 3:
                nonzero_indices_M_A_l = RSU_T + UAV_T + HAP_T + nonzero_indices_M_A
            
            action_index_m = get_next_action_M(state_index_m, nonzero_indices_M_A_l, epsilon_m, M_A, q_values_m)
            new_state_m, new_state_index_m, col_index_m, T_soj_me = Next_State_M(action_index_h, action_index_m, V_spd, Req_Ser, NO_VUs_RSU, Max_U_RSU, service_allocation_RSU, VN_RSU_Soj[v_id], RSU_r, NO_VUs_UAV, Max_U_UAV, service_allocation_UAV, VN_UAV_Soj[v_id], UAV_r, NO_VUs_HAP, Max_U_HAP, service_allocation_HAP, VN_HAP_Soj[v_id], HAP_r, NO_VUs_LEO, Max_U_LEO, service_allocation_LEO, VN_LEO_Soj[v_id], LEO_r)
        
            action_index_l_id = agent_l2.act(state_l, q_values_l)
            
            action_l = Action_Space_L[int(action_index_l_id)]
            out_l = Learning_Cost(VN_Main_IP, DL_Req, T_soj_me, RSU_C_RA, Rate_VR_R, UAV_C_RA, Rate_VU_R, HAP_C_RA, Rate_VH_R, LEO_C_RA, Rate_VL_R, action_index_h, action_index_m, action_index_l_id)
            
            Tot_LC = out_l[0]
            Tot_EC = out_l[1]
            Toc_LC_V = out_l[2]
            Toc_LC_E = out_l[3]
            Toc_EC_V = out_l[4]
            Toc_EC_E = out_l[5]
            F1_V = out_l[6]
            F2_V = out_l[7]
            F3_V = out_l[8]
            
            new_state_l = [0, 0, 0]
            if F1_V > 0:
                new_state_l[0] = 1
            if F2_V > 0:
                new_state_l[1] = 1
            if F3_V > 0:
                new_state_l[0] = 1
            
            indices = np.argwhere((State_Space_L == new_state_l).all(axis=1))
            new_state_index_l = indices[0, 0]
            
            state_index_h = new_state_index_h
            state_index_m = new_state_index_m
            state_index_l = new_state_index_l
            
            state_h = State_Space_H[state_index_h]
            state_m = State_Space_M[state_index_m]
            state_l = State_Space_L[state_index_l]

            
    
        HRL_decisions2[v][0] = action_index_h
        if(action_index_h ==0):
            Node_seelction_choice=action_index_m
            indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
            HRL_decisions2[v][1] = random.choice(indices_with_ones.tolist())
            HRL_decisions2[v][2]=action_l
            
        if(action_index_h ==1):
            Node_seelction_choice=action_index_m
            indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
            HRL_decisions2[v][1] = random.choice(indices_with_ones.tolist())
            HRL_decisions2[v][2]=action_l
    
        if(action_index_h ==2):
            Node_seelction_choice=action_index_m
            indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
            HRL_decisions2[v][1] = random.choice(indices_with_ones.tolist())
            HRL_decisions2[v][2]=action_l
            
        if(action_index_h ==3):
            Node_seelction_choice=action_index_m
            indices_with_ones = np.where(np.array(Node_seelction_choice) == 1)[0]
            HRL_decisions2[v][1] = random.choice(indices_with_ones.tolist())
            HRL_decisions2[v][2]=action_l
    
    
    
    NO_VUs_RSU_HRL2=np.zeros(RSU_T)
    NO_VUs_UAV_HRL2=np.zeros(UAV_T)
    NO_VUs_HAP_HRL2=np.zeros(HAP_T)
    NO_VUs_LEO_HRL2=np.zeros(LEO_T)
    for v in range(V):
        if(HRL_decisions2[v][0]==0):
            for r in range(RSU_T):
                if(HRL_decisions2[v][1]==r):
                    NO_VUs_RSU_HRL2[r]=NO_VUs_RSU_HRL2[r]+1
    
        if(HRL_decisions2[v][0]==1):
            for u in range(UAV_T):
                if(HRL_decisions2[v][1]==u):
                    NO_VUs_UAV_HRL2[u]=NO_VUs_UAV_HRL2[u]+1
    
        if(HRL_decisions2[v][0]==2):
            for h in range(HAP_T):
                if(HRL_decisions2[v][1]==h):
                    NO_VUs_HAP_HRL2[h]=NO_VUs_HAP_HRL2[h]+1
                    
        if(HRL_decisions2[v][0]==3):
            for s in range(LEO_T):
                if(HRL_decisions2[v][1]==s):
                    NO_VUs_LEO_HRL2[s]=NO_VUs_LEO_HRL2[s]+1
    
    
    #-----Resource Allocation------------
    RA_HRL2_OUT= Resource_Allocation( VN_Main_IP,  HRL_decisions2, NO_VUs_RSU_HRL2, NO_VUs_UAV_HRL2, NO_VUs_HAP_HRL2, NO_VUs_LEO_HRL2)
    
    RSU_B_HRL2=RA_HRL2_OUT[0]
    UAV_B_HRL2=RA_HRL2_OUT[1]
    HAP_B_HRL2=RA_HRL2_OUT[2]
    LEO_B_HRL2=RA_HRL2_OUT[3]
    RSU_C_HRL2=RA_HRL2_OUT[4]
    UAV_C_HRL2=RA_HRL2_OUT[5]
    HAP_C_HRL2=RA_HRL2_OUT[6]
    LEO_C_HRL2=RA_HRL2_OUT[7]
    
    Rate_VR_HRL2, Rate_VU_HRL2, Rate_VH_HRL2, Rate_VL_HRL2= Data_Rate(VN_Main_IP,HRL_decisions2, RSU_B_HRL2, UAV_B_HRL2, HAP_B_HRL2, LEO_B_HRL2)
        
    Cost_HRL2=Task_Proc_Main(VN_Main_IP, RSU_C_HRL2, Rate_VR_HRL2, UAV_C_HRL2, Rate_VU_HRL2, HAP_C_HRL2, Rate_VH_HRL2, LEO_C_HRL2, Rate_VL_HRL2, 0.25, 0.75, HRL_decisions2)
    T_L_HRL2=Cost_HRL2[0]
    T_E_HRL2=Cost_HRL2[1]
    T_L_V_HRL2=Cost_HRL2[2]
    T_L_E_HRL2=Cost_HRL2[3]
    T_E_V_HRL2=Cost_HRL2[4]
    T_E_E_HRL2=Cost_HRL2[5]
    TC_HRL2=Cost_HRL2[8]
    No_of_Ser_L_Fail_HRL2[V_it]=Cost_HRL2[9]
    No_of_Soj_L_Fail_HRL2[V_it]=Cost_HRL2[10]
    No_of_Ser_H_Req_HRL2[V_it]=Cost_HRL2[11]
    

    Tot_TC_HRL2[V_it]=np.mean(TC_HRL2)
    Tot_T_L_HRL2[V_it]=np.mean(T_L_HRL2)
    Tot_T_E_HRL2[V_it]=np.mean(T_E_HRL2)
    Tot_T_L_V_HRL2[V_it]=np.mean(T_L_V_HRL2)
    Tot_T_L_E_HRL2[V_it]=np.mean(T_L_E_HRL2)
    Tot_T_E_V_HRL2[V_it]=np.mean(T_E_V_HRL2)
    Tot_T_E_E_HRL2[V_it]=np.mean(T_E_E_HRL2)

    Tot_TC_SD_HRL2[V_it]=np.std(TC_HRL2)
    Tot_T_L_SD_HRL2[V_it]=np.std(T_L_HRL2)
    Tot_T_E_SD_HRL2[V_it]=np.std(T_E_HRL2)
    Tot_T_L_V_SD_HRL2[V_it]=np.std(T_L_V_HRL2)
    Tot_T_L_E_SD_HRL2[V_it]=np.std(T_L_E_HRL2)
    Tot_T_E_V_SD_HRL2[V_it]=np.std(T_E_V_HRL2)
    Tot_T_E_E_SD_HRL2[V_it]=np.std(T_E_E_HRL2)



#------------Local Computing--------------------------
    T_loc_LC=Cost_HRL[4]
    E_loc_LC=Cost_HRL[5]
    TC_LC_n=np.zeros((1,V))
    No_Ser_F=0

    Tot_T_L_LC[V_it]=np.mean(T_loc_LC)
    Tot_T_E_LC[V_it]=np.mean(E_loc_LC)
    Tot_T_L_V_LC[V_it]=np.mean(T_loc_LC)
    Tot_T_L_E_LC[V_it]=0
    Tot_T_E_V_LC[V_it]=np.mean(E_loc_LC)
    Tot_T_E_E_LC[V_it]=0
    for v in range(V):
        TC_LC_n[0][v]=T_loc_LC[0][v] + E_loc_LC[0][v]
        if(T_loc_LC[0][v] > 4):
            No_Ser_F=No_Ser_F +1
            
    Tot_TC_LC[V_it]=np.mean(TC_LC_n)
    No_of_Ser_L_Fail_LC[V_it]=No_Ser_F
    No_of_Soj_L_Fail_LC[V_it]=0
    No_of_Ser_H_Req_LC[V_it]=0

    

    Tot_T_L_SD_LC[V_it]=np.std(T_loc_LC)
    Tot_T_E_SD_LC[V_it]=np.std(E_loc_LC)
    Tot_T_L_V_SD_LC[V_it]=np.std(T_loc_LC)
    Tot_T_L_E_SD_LC[V_it]=0
    Tot_T_E_V_SD_LC[V_it]=np.std(E_loc_LC)
    Tot_T_E_E_SD_LC[V_it]=0

#-------------------------------------------------------

    #---------Distance Based Solutions......................

    DA_decisions=np.zeros((V,3))

    ratios1 = np.divide(VN_RSU_dist_Sj, VN_RSU_dist, out=np.zeros_like(VN_RSU_dist), where=VN_RSU_dist_Sj!=0)
    ratios2 = np.divide(VN_UAV_dist_Sj, VN_UAV_dist, out=np.zeros_like(VN_UAV_dist), where=VN_UAV_dist_Sj!=0)
    ratios3 = np.divide(VN_HAP_dist_Sj, VN_HAP_dist, out=np.zeros_like(VN_HAP_dist), where=VN_HAP_dist_Sj!=0)
    ratios4 = np.divide(VN_LEO_dist_Sj, VN_LEO_dist, out=np.zeros_like(VN_LEO_dist), where=VN_LEO_dist_Sj!=0)
    
    max_values1 = np.max(ratios1, axis=1)
    max_indices1 = np.argmax(ratios1, axis=1)
    max_values2 = np.max(ratios2, axis=1)
    max_indices2 = np.argmax(ratios2, axis=1)
    max_values3 = np.max(ratios3, axis=1)
    max_indices3 = np.argmax(ratios3, axis=1)
    max_values4 = np.max(ratios4, axis=1)
    max_indices4 = np.argmax(ratios4, axis=1)
    
    combined_max_values = np.stack((max_values1, max_values2, max_values3, max_values4), axis=1)
    combined_max_indices = np.stack((max_indices1, max_indices2, max_indices3, max_indices4), axis=1)
    overall_max_values = np.max(combined_max_values, axis=1)
    overall_max_sources = np.argmax(combined_max_values, axis=1)
    final_indices = combined_max_indices[np.arange(combined_max_indices.shape[0]), overall_max_sources]
    
    for v in range(V):
        DA_decisions[v][0]=overall_max_sources[v]
        DA_decisions[v][1]=final_indices[v]
        DA_decisions[v][2]= random.random()


    NO_VUs_RSU_DA=np.zeros(RSU_T)
    NO_VUs_UAV_DA=np.zeros(UAV_T)
    NO_VUs_HAP_DA=np.zeros(HAP_T)
    NO_VUs_LEO_DA=np.zeros(LEO_T)
    for v in range(V):
        if(DA_decisions[v][0]==0):
            for r in range(RSU_T):
                if(DA_decisions[v][1]==r):
                    NO_VUs_RSU_DA[r]=NO_VUs_RSU_DA[r]+1
    
        if(DA_decisions[v][0]==1):
            for u in range(UAV_T):
                if(DA_decisions[v][1]==u):
                    NO_VUs_UAV_DA[u]=NO_VUs_UAV_DA[u]+1
    
        if(DA_decisions[v][0]==2):
            for h in range(HAP_T):
                if(DA_decisions[v][1]==h):
                    NO_VUs_HAP_DA[h]=NO_VUs_HAP_DA[h]+1
                    
        if(DA_decisions[v][0]==3):
            for s in range(LEO_T):
                if(DA_decisions[v][1]==s):
                    NO_VUs_LEO_DA[s]=NO_VUs_LEO_DA[s]+1    

    #-----Resource Allocation------------
    RA_DA_OUT= Resource_Allocation( VN_Main_IP,  DA_decisions, NO_VUs_RSU_DA, NO_VUs_UAV_DA, NO_VUs_HAP_DA, NO_VUs_LEO_DA)
    
    RSU_B_DA=RA_DA_OUT[0]
    UAV_B_DA=RA_DA_OUT[1]
    HAP_B_DA=RA_DA_OUT[2]
    LEO_B_DA=RA_DA_OUT[3]
    RSU_C_DA=RA_DA_OUT[4]
    UAV_C_DA=RA_DA_OUT[5]
    HAP_C_DA=RA_DA_OUT[6]
    LEO_C_DA=RA_DA_OUT[7]
    
    Rate_VR_DA, Rate_VU_DA, Rate_VH_DA, Rate_VL_DA= Data_Rate(VN_Main_IP,DA_decisions, RSU_B_DA, UAV_B_DA, HAP_B_DA, LEO_B_DA)
        
    Cost_DA=Task_Proc_Main(VN_Main_IP, RSU_C_DA, Rate_VR_DA, UAV_C_DA, Rate_VU_DA, HAP_C_DA, Rate_VH_DA, LEO_C_DA, Rate_VL_DA, 0.5, 0.5, DA_decisions)
    T_L_DA=Cost_DA[0]
    T_E_DA=Cost_DA[1]
    T_L_V_DA=Cost_DA[2]
    T_L_E_DA=Cost_DA[3]
    T_E_V_DA=Cost_DA[4]
    T_E_E_DA=Cost_DA[5]
    TC_DA=Cost_DA[8]
    No_of_Ser_L_Fail_DA[V_it]=Cost_DA[9]
    No_of_Soj_L_Fail_DA[V_it]=Cost_DA[10]
    No_of_Ser_H_Req_DA[V_it]=Cost_DA[11]

    Tot_TC_DA[V_it]=np.mean(TC_DA)
    Tot_T_L_DA[V_it]=np.mean(T_L_DA)
    Tot_T_E_DA[V_it]=np.mean(T_E_DA)
    Tot_T_L_V_DA[V_it]=np.mean(T_L_V_DA)
    Tot_T_L_E_DA[V_it]=np.mean(T_L_E_DA)
    Tot_T_E_V_DA[V_it]=np.mean(T_E_V_DA)
    Tot_T_E_E_DA[V_it]=np.mean(T_E_E_DA)

    Tot_TC_SD_DA[V_it]=np.std(TC_HRL)
    Tot_T_L_SD_DA[V_it]=np.std(T_L_DA)
    Tot_T_E_SD_DA[V_it]=np.std(T_E_DA)
    Tot_T_L_V_SD_DA[V_it]=np.std(T_L_V_DA)
    Tot_T_L_E_SD_DA[V_it]=np.std(T_L_E_DA)
    Tot_T_E_V_SD_DA[V_it]=np.std(T_E_V_DA)
    Tot_T_E_E_SD_DA[V_it]=np.std(T_E_E_DA)




In [ ]:
data_i = np.array(Tot_V)
Tot_V_n=data_i.reshape(-1, RP)
Users = 10*Tot_V_n.mean(axis=1)

Tot_TC_RA_n = Tot_TC_RA.reshape(-1, RP)
Tot_T_L_RA_n = Tot_T_L_RA.reshape(-1, RP)
Tot_T_E_RA_n = Tot_T_E_RA.reshape(-1, RP)
No_of_Ser_L_Fail_RA_n = No_of_Ser_L_Fail_RA.reshape(-1, RP)
No_of_Soj_L_Fail_RA_n = No_of_Soj_L_Fail_RA.reshape(-1, RP)
No_of_Ser_H_Req_RA_n = No_of_Ser_H_Req_RA.reshape(-1, RP)

Tot_TC_SD_RA_n = Tot_TC_SD_RA.reshape(-1, RP)
Tot_T_L_SD_RA_n = Tot_T_L_SD_RA.reshape(-1, RP)
Tot_T_E_SD_RA_n = Tot_T_E_SD_RA.reshape(-1, RP)

Tot_TC_RA_nn = Tot_TC_RA_n.mean(axis=1)
Tot_T_L_RA_nn = Tot_T_L_RA_n.mean(axis=1)
Tot_T_E_RA_nn = Tot_T_E_RA_n.mean(axis=1)
No_of_Ser_L_Fail_RA_nn = No_of_Ser_L_Fail_RA_n.mean(axis=1)
No_of_Soj_L_Fail_RA_nn = No_of_Soj_L_Fail_RA_n.mean(axis=1)
No_of_Ser_H_Req_RA_nn = No_of_Ser_H_Req_RA_n.mean(axis=1)

Tot_TC_SD_RA_nn = Tot_TC_SD_RA_n.mean(axis=1)
Tot_T_L_SD_RA_nn = Tot_T_L_SD_RA_n.mean(axis=1)
Tot_T_E_SD_RA_nn = Tot_T_E_SD_RA_n.mean(axis=1)

#----------------------------------------------------

Tot_TC_HRL_n = Tot_TC_HRL.reshape(-1, RP)
Tot_T_L_HRL_n = Tot_T_L_HRL.reshape(-1, RP)
Tot_T_E_HRL_n = Tot_T_E_HRL.reshape(-1, RP)
No_of_Ser_L_Fail_HRL_n = No_of_Ser_L_Fail_HRL.reshape(-1, RP)
No_of_Soj_L_Fail_HRL_n = No_of_Soj_L_Fail_HRL.reshape(-1, RP)
No_of_Ser_H_Req_HRL_n = No_of_Ser_H_Req_HRL.reshape(-1, RP)

Tot_TC_SD_HRL_n = Tot_TC_SD_HRL.reshape(-1, RP)
Tot_T_L_SD_HRL_n = Tot_T_L_SD_HRL.reshape(-1, RP)
Tot_T_E_SD_HRL_n = Tot_T_E_SD_HRL.reshape(-1, RP)

Tot_TC_HRL_nn = Tot_TC_HRL_n.mean(axis=1)
Tot_T_L_HRL_nn = Tot_T_L_HRL_n.mean(axis=1)
Tot_T_E_HRL_nn = Tot_T_E_HRL_n.mean(axis=1)
No_of_Ser_L_Fail_HRL_nn = No_of_Ser_L_Fail_HRL_n.mean(axis=1)
No_of_Soj_L_Fail_HRL_nn = No_of_Soj_L_Fail_HRL_n.mean(axis=1)
No_of_Ser_H_Req_HRL_nn = No_of_Ser_H_Req_HRL_n.mean(axis=1)

Tot_TC_SD_HRL_nn = Tot_TC_SD_HRL_n.mean(axis=1)
Tot_T_L_SD_HRL_nn = Tot_T_L_SD_HRL_n.mean(axis=1)
Tot_T_E_SD_HRL_nn = Tot_T_E_SD_HRL_n.mean(axis=1)

#----------------------------------------------------
Tot_TC_HRL1_n = Tot_TC_HRL1.reshape(-1, RP)
Tot_T_L_HRL1_n = Tot_T_L_HRL1.reshape(-1, RP)
Tot_T_E_HRL1_n = Tot_T_E_HRL1.reshape(-1, RP)
No_of_Ser_L_Fail_HRL1_n = No_of_Ser_L_Fail_HRL1.reshape(-1, RP)
No_of_Soj_L_Fail_HRL1_n = No_of_Soj_L_Fail_HRL1.reshape(-1, RP)
No_of_Ser_H_Req_HRL1_n = No_of_Ser_H_Req_HRL1.reshape(-1, RP)

Tot_TC_SD_HRL1_n = Tot_TC_SD_HRL1.reshape(-1, RP)
Tot_T_L_SD_HRL1_n = Tot_T_L_SD_HRL1.reshape(-1, RP)
Tot_T_E_SD_HRL1_n = Tot_T_E_SD_HRL1.reshape(-1, RP)

Tot_TC_HRL1_nn = Tot_TC_HRL1_n.mean(axis=1)
Tot_T_L_HRL1_nn = Tot_T_L_HRL1_n.mean(axis=1)
Tot_T_E_HRL1_nn = Tot_T_E_HRL1_n.mean(axis=1)
No_of_Ser_L_Fail_HRL1_nn = No_of_Ser_L_Fail_HRL1_n.mean(axis=1)
No_of_Soj_L_Fail_HRL1_nn = No_of_Soj_L_Fail_HRL1_n.mean(axis=1)
No_of_Ser_H_Req_HRL1_nn = No_of_Ser_H_Req_HRL1_n.mean(axis=1)

Tot_TC_SD_HRL1_nn = Tot_TC_SD_HRL1_n.mean(axis=1)
Tot_T_L_SD_HRL1_nn = Tot_T_L_SD_HRL1_n.mean(axis=1)
Tot_T_E_SD_HRL1_nn = Tot_T_E_SD_HRL1_n.mean(axis=1)

#----------------------------------------------------
Tot_TC_HRL2_n = Tot_TC_HRL2.reshape(-1, RP)
Tot_T_L_HRL2_n = Tot_T_L_HRL2.reshape(-1, RP)
Tot_T_E_HRL2_n = Tot_T_E_HRL2.reshape(-1, RP)
No_of_Ser_L_Fail_HRL2_n = No_of_Ser_L_Fail_HRL2.reshape(-1, RP)
No_of_Soj_L_Fail_HRL2_n = No_of_Soj_L_Fail_HRL2.reshape(-1, RP)
No_of_Ser_H_Req_HRL2_n = No_of_Ser_H_Req_HRL2.reshape(-1, RP)

Tot_TC_SD_HRL2_n = Tot_TC_SD_HRL2.reshape(-1, RP)
Tot_T_L_SD_HRL2_n = Tot_T_L_SD_HRL2.reshape(-1, RP)
Tot_T_E_SD_HRL2_n = Tot_T_E_SD_HRL2.reshape(-1, RP)

Tot_TC_HRL2_nn = Tot_TC_HRL2_n.mean(axis=1)
Tot_T_L_HRL2_nn = Tot_T_L_HRL2_n.mean(axis=1)
Tot_T_E_HRL2_nn = Tot_T_E_HRL2_n.mean(axis=1)
No_of_Ser_L_Fail_HRL2_nn = No_of_Ser_L_Fail_HRL2_n.mean(axis=1)
No_of_Soj_L_Fail_HRL2_nn = No_of_Soj_L_Fail_HRL2_n.mean(axis=1)
No_of_Ser_H_Req_HRL2_nn = No_of_Ser_H_Req_HRL2_n.mean(axis=1)

Tot_TC_SD_HRL2_nn = Tot_TC_SD_HRL2_n.mean(axis=1)
Tot_T_L_SD_HRL2_nn = Tot_T_L_SD_HRL2_n.mean(axis=1)
Tot_T_E_SD_HRL2_nn = Tot_T_E_SD_HRL2_n.mean(axis=1)

#----------------------------------------------------------
Tot_TC_LC_n = Tot_TC_LC.reshape(-1, RP)
Tot_T_L_LC_n = Tot_T_L_LC.reshape(-1, RP)
Tot_T_E_LC_n = Tot_T_E_LC.reshape(-1, RP)
No_of_Ser_L_Fail_LC_n = No_of_Ser_L_Fail_LC.reshape(-1, RP)
No_of_Soj_L_Fail_LC_n = No_of_Soj_L_Fail_LC.reshape(-1, RP)
No_of_Ser_H_Req_LC_n = No_of_Ser_H_Req_LC.reshape(-1, RP)

Tot_TC_SD_LC_n = Tot_TC_SD_LC.reshape(-1, RP)
Tot_T_L_SD_LC_n = Tot_T_L_SD_LC.reshape(-1, RP)
Tot_T_E_SD_LC_n = Tot_T_E_SD_LC.reshape(-1, RP)

Tot_TC_LC_nn = Tot_TC_LC_n.mean(axis=1)
Tot_T_L_LC_nn = Tot_T_L_LC_n.mean(axis=1)
Tot_T_E_LC_nn = Tot_T_E_LC_n.mean(axis=1)
No_of_Ser_L_Fail_LC_nn = No_of_Ser_L_Fail_LC_n.mean(axis=1)
No_of_Soj_L_Fail_LC_nn = No_of_Soj_L_Fail_LC_n.mean(axis=1)
No_of_Ser_H_Req_LC_nn = No_of_Ser_H_Req_LC_n.mean(axis=1)

Tot_TC_SD_LC_nn = Tot_TC_SD_LC_n.mean(axis=1)
Tot_T_L_SD_LC_nn = Tot_T_L_SD_LC_n.mean(axis=1)
Tot_T_E_SD_LC_nn = Tot_T_E_SD_LC_n.mean(axis=1)

#-------------------------------------------------------
Tot_TC_DA_n = Tot_TC_DA.reshape(-1, RP)
Tot_T_L_DA_n = Tot_T_L_DA.reshape(-1, RP)
Tot_T_E_DA_n = Tot_T_E_DA.reshape(-1, RP)
No_of_Ser_L_Fail_DA_n = No_of_Ser_L_Fail_DA.reshape(-1, RP)
No_of_Soj_L_Fail_DA_n = No_of_Soj_L_Fail_DA.reshape(-1, RP)
No_of_Ser_H_Req_DA_n = No_of_Ser_H_Req_DA.reshape(-1, RP)

Tot_TC_SD_DA_n = Tot_TC_SD_DA.reshape(-1, RP)
Tot_T_L_SD_DA_n = Tot_T_L_SD_DA.reshape(-1, RP)
Tot_T_E_SD_DA_n = Tot_T_E_SD_DA.reshape(-1, RP)

Tot_TC_DA_nn = Tot_TC_DA_n.mean(axis=1)
Tot_T_L_DA_nn = Tot_T_L_DA_n.mean(axis=1)
Tot_T_E_DA_nn = Tot_T_E_DA_n.mean(axis=1)
No_of_Ser_L_Fail_DA_nn = No_of_Ser_L_Fail_DA_n.mean(axis=1)
No_of_Soj_L_Fail_DA_nn = No_of_Soj_L_Fail_DA_n.mean(axis=1)
No_of_Ser_H_Req_DA_nn = No_of_Ser_H_Req_DA_n.mean(axis=1)

Tot_TC_SD_DA_nn = Tot_TC_SD_DA_n.mean(axis=1)
Tot_T_L_SD_DA_nn = Tot_T_L_SD_DA_n.mean(axis=1)
Tot_T_E_SD_DA_nn = Tot_T_E_SD_DA_n.mean(axis=1)

Req_LC=4*np.ones(len(Tot_T_E_SD_DA_nn))

In [ ]:
#---Uncomment to see new results  ---------------
# df1 = pd.DataFrame({'Tot_TC_HRL2_nn': Tot_TC_HRL2_nn})
# df2 = pd.DataFrame({'Tot_T_L_HRL2_nn': Tot_T_L_HRL2_nn})
# df3 = pd.DataFrame({'Tot_T_E_HRL2_nn': Tot_T_E_HRL2_nn})
# df4 = pd.DataFrame({'No_of_Ser_L_Fail_HRL2_nn': No_of_Ser_L_Fail_HRL2_nn})
# df5 = pd.DataFrame({'No_of_Soj_L_Fail_HRL2_nn': No_of_Soj_L_Fail_HRL2_nn})
# df6 = pd.DataFrame({'No_of_Ser_H_Req_HRL2_nn': No_of_Ser_H_Req_HRL2_nn})
# df7 = pd.DataFrame({'Tot_TC_SD_HRL2_nn': Tot_TC_SD_HRL2_nn})
# df8 = pd.DataFrame({'Tot_T_L_SD_HRL2_nn': Tot_T_L_SD_HRL2_nn})
# df9 = pd.DataFrame({'Tot_T_E_SD_HRL2_nn': Tot_T_E_SD_HRL2_nn})
# df10 = pd.DataFrame({'Tot_TC_HRL1_nn': Tot_TC_HRL1_nn})
# df11 = pd.DataFrame({'Tot_T_L_HRL1_nn': Tot_T_L_HRL1_nn})
# df12 = pd.DataFrame({'Tot_T_E_HRL1_nn': Tot_T_E_HRL1_nn})
# df13 = pd.DataFrame({'No_of_Ser_L_Fail_HRL1_nn': No_of_Ser_L_Fail_HRL1_nn})
# df14 = pd.DataFrame({'No_of_Soj_L_Fail_HRL1_nn': No_of_Soj_L_Fail_HRL1_nn})
# df15 = pd.DataFrame({'No_of_Ser_H_Req_HRL1_nn': No_of_Ser_H_Req_HRL1_nn})
# df16 = pd.DataFrame({'Tot_TC_SD_HRL1_nn': Tot_TC_SD_HRL1_nn})
# df17 = pd.DataFrame({'Tot_T_L_SD_HRL1_nn': Tot_T_L_SD_HRL1_nn})
# df18 = pd.DataFrame({'Tot_T_E_SD_HRL1_nn': Tot_T_E_SD_HRL1_nn})
# df19 = pd.DataFrame({'Tot_TC_HRL_nn': Tot_TC_HRL_nn})
# df20 = pd.DataFrame({'Tot_T_L_HRL_nn': Tot_T_L_HRL_nn})
# df21 = pd.DataFrame({'Tot_T_E_HRL_nn': Tot_T_E_HRL_nn})
# df22 = pd.DataFrame({'No_of_Ser_L_Fail_HRL_nn': No_of_Ser_L_Fail_HRL_nn})
# df23 = pd.DataFrame({'No_of_Soj_L_Fail_HRL_nn': No_of_Soj_L_Fail_HRL_nn})
# df24 = pd.DataFrame({'No_of_Ser_H_Req_HRL_nn': No_of_Ser_H_Req_HRL_nn})
# df25 = pd.DataFrame({'Tot_TC_SD_HRL_nn': Tot_TC_SD_HRL_nn})
# df26 = pd.DataFrame({'Tot_T_L_SD_HRL_nn': Tot_T_L_SD_HRL_nn})
# df27 = pd.DataFrame({'Tot_T_E_SD_HRL_nn': Tot_T_E_SD_HRL_nn})
# df28 = pd.DataFrame({'Tot_TC_DA_nn': Tot_TC_DA_nn})
# df29 = pd.DataFrame({'Tot_T_L_DA_nn': Tot_T_L_DA_nn})
# df30 = pd.DataFrame({'Tot_T_E_DA_nn': Tot_T_E_DA_nn})
# df31 = pd.DataFrame({'No_of_Ser_L_Fail_DA_nn': No_of_Ser_L_Fail_DA_nn})
# df32 = pd.DataFrame({'No_of_Soj_L_Fail_DA_nn': No_of_Soj_L_Fail_DA_nn})
# df33 = pd.DataFrame({'No_of_Ser_H_Req_DA_nn': No_of_Ser_H_Req_DA_nn})
# df34 = pd.DataFrame({'Tot_TC_SD_DA_nn': Tot_TC_SD_DA_nn})
# df35 = pd.DataFrame({'Tot_T_L_SD_DA_nn': Tot_T_L_SD_DA_nn})
# df36 = pd.DataFrame({'Tot_T_E_SD_DA_nn': Tot_T_E_SD_DA_nn})
# df37 = pd.DataFrame({'Tot_TC_RA_nn': Tot_TC_RA_nn})
# df38 = pd.DataFrame({'Tot_T_L_RA_nn': Tot_T_L_RA_nn})
# df39 = pd.DataFrame({'Tot_T_E_RA_nn': Tot_T_E_RA_nn})
# df40 = pd.DataFrame({'No_of_Ser_L_Fail_RA_nn': No_of_Ser_L_Fail_RA_nn})
# df41 = pd.DataFrame({'No_of_Soj_L_Fail_RA_nn': No_of_Soj_L_Fail_RA_nn})
# df42 = pd.DataFrame({'No_of_Ser_H_Req_RA_nn': No_of_Ser_H_Req_RA_nn})
# df43 = pd.DataFrame({'Tot_TC_SD_RA_nn': Tot_TC_SD_RA_nn})
# df44 = pd.DataFrame({'Tot_T_L_SD_RA_nn': Tot_T_L_SD_RA_nn})
# df45 = pd.DataFrame({'Tot_T_E_SD_RA_nn': Tot_T_E_SD_RA_nn})
# df46 = pd.DataFrame({'Tot_TC_LC_nn': Tot_TC_LC_nn})
# df47 = pd.DataFrame({'Tot_T_L_LC_nn': Tot_T_L_LC_nn})
# df48 = pd.DataFrame({'Tot_T_E_LC_nn': Tot_T_E_LC_nn})
# df49 = pd.DataFrame({'No_of_Ser_L_Fail_LC_nn': No_of_Ser_L_Fail_LC_nn})
# df50 = pd.DataFrame({'No_of_Soj_L_Fail_LC_nn': No_of_Soj_L_Fail_LC_nn})
# df51 = pd.DataFrame({'No_of_Ser_H_Req_LC_nn': No_of_Ser_H_Req_LC_nn})
# df52 = pd.DataFrame({'Tot_TC_SD_LC_nn': Tot_TC_SD_LC_nn})
# df53 = pd.DataFrame({'Tot_T_L_SD_LC_nn': Tot_T_L_SD_LC_nn})
# df54 = pd.DataFrame({'Tot_T_E_SD_LC_nn': Tot_T_E_SD_LC_nn})

# excel_file = 'result.xlsx'
# with pd.ExcelWriter(excel_file) as writer:
#     df1.to_excel(writer, sheet_name='Tot_TC_HRL2_nn', index=False)
#     df2.to_excel(writer, sheet_name='Tot_T_L_HRL2_nn', index=False)
#     df3.to_excel(writer, sheet_name='Tot_T_E_HRL2_nn', index=False)
#     df4.to_excel(writer, sheet_name='No_of_Ser_L_Fail_HRL2_nn', index=False)
#     df5.to_excel(writer, sheet_name='No_of_Soj_L_Fail_HRL2_nn', index=False)
#     df6.to_excel(writer, sheet_name='No_of_Ser_H_Req_HRL2_nn', index=False)
#     df7.to_excel(writer, sheet_name='Tot_TC_SD_HRL2_nn', index=False)
#     df8.to_excel(writer, sheet_name='Tot_T_L_SD_HRL2_nn', index=False)
#     df9.to_excel(writer, sheet_name='Tot_T_E_SD_HRL2_nn', index=False)
#     df10.to_excel(writer, sheet_name='Tot_TC_HRL1_nn', index=False)
#     df11.to_excel(writer, sheet_name='Tot_T_L_HRL1_nn', index=False)
#     df12.to_excel(writer, sheet_name='Tot_T_E_HRL1_nn', index=False)
#     df13.to_excel(writer, sheet_name='No_of_Ser_L_Fail_HRL1_nn', index=False)
#     df14.to_excel(writer, sheet_name='No_of_Soj_L_Fail_HRL1_nn', index=False)
#     df15.to_excel(writer, sheet_name='No_of_Ser_H_Req_HRL1_nn', index=False)
#     df16.to_excel(writer, sheet_name='Tot_TC_SD_HRL1_nn', index=False)
#     df17.to_excel(writer, sheet_name='Tot_T_L_SD_HRL1_nn', index=False)
#     df18.to_excel(writer, sheet_name='Tot_T_E_SD_HRL1_nn', index=False)
#     df19.to_excel(writer, sheet_name='Tot_TC_HRL_nn', index=False)
#     df20.to_excel(writer, sheet_name='Tot_T_L_HRL_nn', index=False)
#     df21.to_excel(writer, sheet_name='Tot_T_E_HRL_nn', index=False)
#     df22.to_excel(writer, sheet_name='No_of_Ser_L_Fail_HRL_nn', index=False)
#     df23.to_excel(writer, sheet_name='No_of_Soj_L_Fail_HRL_nn', index=False)
#     df24.to_excel(writer, sheet_name='No_of_Ser_H_Req_HRL_nn', index=False)
#     df25.to_excel(writer, sheet_name='Tot_TC_SD_HRL_nn', index=False)
#     df26.to_excel(writer, sheet_name='Tot_T_L_SD_HRL_nn', index=False)
#     df27.to_excel(writer, sheet_name='Tot_T_E_SD_HRL_nn', index=False)
#     df28.to_excel(writer, sheet_name='Tot_TC_DA_nn', index=False)
#     df29.to_excel(writer, sheet_name='Tot_T_L_DA_nn', index=False)
#     df30.to_excel(writer, sheet_name='Tot_T_E_DA_nn', index=False)
#     df31.to_excel(writer, sheet_name='No_of_Ser_L_Fail_DA_nn', index=False)
#     df32.to_excel(writer, sheet_name='No_of_Soj_L_Fail_DA_nn', index=False)
#     df33.to_excel(writer, sheet_name='No_of_Ser_H_Req_DA_nn', index=False)
#     df34.to_excel(writer, sheet_name='Tot_TC_SD_DA_nn', index=False)
#     df35.to_excel(writer, sheet_name='Tot_T_L_SD_DA_nn', index=False)
#     df36.to_excel(writer, sheet_name='Tot_T_E_SD_DA_nn', index=False)
#     df37.to_excel(writer, sheet_name='Tot_TC_RA_nn', index=False)
#     df38.to_excel(writer, sheet_name='Tot_T_L_RA_nn', index=False)
#     df39.to_excel(writer, sheet_name='Tot_T_E_RA_nn', index=False)
#     df40.to_excel(writer, sheet_name='No_of_Ser_L_Fail_RA_nn', index=False)
#     df41.to_excel(writer, sheet_name='No_of_Soj_L_Fail_RA_nn', index=False)
#     df42.to_excel(writer, sheet_name='No_of_Ser_H_Req_RA_nn', index=False)
#     df43.to_excel(writer, sheet_name='Tot_TC_SD_RA_nn', index=False)
#     df44.to_excel(writer, sheet_name='Tot_T_L_SD_RA_nn', index=False)
#     df45.to_excel(writer, sheet_name='Tot_T_E_SD_RA_nn', index=False)
#     df46.to_excel(writer, sheet_name='Tot_TC_LC_nn', index=False)
#     df47.to_excel(writer, sheet_name='Tot_T_L_LC_nn', index=False)
#     df48.to_excel(writer, sheet_name='Tot_T_E_LC_nn', index=False)
#     df49.to_excel(writer, sheet_name='No_of_Ser_L_Fail_LC_nn', index=False)
#     df50.to_excel(writer, sheet_name='No_of_Soj_L_Fail_LC_nn', index=False)
#     df51.to_excel(writer, sheet_name='No_of_Ser_H_Req_LC_nn', index=False)
#     df52.to_excel(writer, sheet_name='Tot_TC_SD_LC_nn', index=False)
#     df53.to_excel(writer, sheet_name='Tot_T_L_SD_LC_nn', index=False)
#     df54.to_excel(writer, sheet_name='Tot_T_E_SD_LC_nn', index=False)



In [ ]:

excel_file = 'result.xlsx'
read_df1 = pd.read_excel(excel_file, sheet_name='Tot_TC_HRL2_nn')
read_df2 = pd.read_excel(excel_file, sheet_name='Tot_T_L_HRL2_nn')
read_df3 = pd.read_excel(excel_file, sheet_name='Tot_T_E_HRL2_nn')
read_df4 = pd.read_excel(excel_file, sheet_name='No_of_Ser_L_Fail_HRL2_nn')
read_df5 = pd.read_excel(excel_file, sheet_name='No_of_Soj_L_Fail_HRL2_nn')
read_df6 = pd.read_excel(excel_file, sheet_name='No_of_Ser_H_Req_HRL2_nn')
read_df7 = pd.read_excel(excel_file, sheet_name='Tot_TC_SD_HRL2_nn')
read_df8 = pd.read_excel(excel_file, sheet_name='Tot_T_L_SD_HRL2_nn')
read_df9 = pd.read_excel(excel_file, sheet_name='Tot_T_E_SD_HRL2_nn')
read_df10 = pd.read_excel(excel_file, sheet_name='Tot_TC_HRL1_nn')
read_df11 = pd.read_excel(excel_file, sheet_name='Tot_T_L_HRL1_nn')
read_df12 = pd.read_excel(excel_file, sheet_name='Tot_T_E_HRL1_nn')
read_df13 = pd.read_excel(excel_file, sheet_name='No_of_Ser_L_Fail_HRL1_nn')
read_df14 = pd.read_excel(excel_file, sheet_name='No_of_Soj_L_Fail_HRL1_nn')
read_df15 = pd.read_excel(excel_file, sheet_name='No_of_Ser_H_Req_HRL1_nn')
read_df16 = pd.read_excel(excel_file, sheet_name='Tot_TC_SD_HRL1_nn')
read_df17 = pd.read_excel(excel_file, sheet_name='Tot_T_L_SD_HRL1_nn')
read_df18 = pd.read_excel(excel_file, sheet_name='Tot_T_E_SD_HRL1_nn')
read_df19 = pd.read_excel(excel_file, sheet_name='Tot_TC_HRL_nn')
read_df20 = pd.read_excel(excel_file, sheet_name='Tot_T_L_HRL_nn')
read_df21 = pd.read_excel(excel_file, sheet_name='Tot_T_E_HRL_nn')
read_df22 = pd.read_excel(excel_file, sheet_name='No_of_Ser_L_Fail_HRL_nn')
read_df23 = pd.read_excel(excel_file, sheet_name='No_of_Soj_L_Fail_HRL_nn')
read_df24 = pd.read_excel(excel_file, sheet_name='No_of_Ser_H_Req_HRL_nn')
read_df25 = pd.read_excel(excel_file, sheet_name='Tot_TC_SD_HRL_nn')
read_df26 = pd.read_excel(excel_file, sheet_name='Tot_T_L_SD_HRL_nn')
read_df27 = pd.read_excel(excel_file, sheet_name='Tot_T_E_SD_HRL_nn')
read_df28 = pd.read_excel(excel_file, sheet_name='Tot_TC_DA_nn')
read_df29 = pd.read_excel(excel_file, sheet_name='Tot_T_L_DA_nn')
read_df30 = pd.read_excel(excel_file, sheet_name='Tot_T_E_DA_nn')
read_df31 = pd.read_excel(excel_file, sheet_name='No_of_Ser_L_Fail_DA_nn')
read_df32 = pd.read_excel(excel_file, sheet_name='No_of_Soj_L_Fail_DA_nn')
read_df33 = pd.read_excel(excel_file, sheet_name='No_of_Ser_H_Req_DA_nn')
read_df34 = pd.read_excel(excel_file, sheet_name='Tot_TC_SD_DA_nn')
read_df35 = pd.read_excel(excel_file, sheet_name='Tot_T_L_SD_DA_nn')
read_df36 = pd.read_excel(excel_file, sheet_name='Tot_T_E_SD_DA_nn')
read_df37 = pd.read_excel(excel_file, sheet_name='Tot_TC_RA_nn')
read_df38 = pd.read_excel(excel_file, sheet_name='Tot_T_L_RA_nn')
read_df39 = pd.read_excel(excel_file, sheet_name='Tot_T_E_RA_nn')
read_df40 = pd.read_excel(excel_file, sheet_name='No_of_Ser_L_Fail_RA_nn')
read_df41 = pd.read_excel(excel_file, sheet_name='No_of_Soj_L_Fail_RA_nn')
read_df42 = pd.read_excel(excel_file, sheet_name='No_of_Ser_H_Req_RA_nn')
read_df43 = pd.read_excel(excel_file, sheet_name='Tot_TC_SD_RA_nn')
read_df44 = pd.read_excel(excel_file, sheet_name='Tot_T_L_SD_RA_nn')
read_df45 = pd.read_excel(excel_file, sheet_name='Tot_T_E_SD_RA_nn')
read_df46 = pd.read_excel(excel_file, sheet_name='Tot_TC_LC_nn')
read_df47 = pd.read_excel(excel_file, sheet_name='Tot_T_L_LC_nn')
read_df48 = pd.read_excel(excel_file, sheet_name='Tot_T_E_LC_nn')
read_df49 = pd.read_excel(excel_file, sheet_name='No_of_Ser_L_Fail_LC_nn')
read_df50 = pd.read_excel(excel_file, sheet_name='No_of_Soj_L_Fail_LC_nn')
read_df51 = pd.read_excel(excel_file, sheet_name='No_of_Ser_H_Req_LC_nn')
read_df52 = pd.read_excel(excel_file, sheet_name='Tot_TC_SD_LC_nn')
read_df53 = pd.read_excel(excel_file, sheet_name='Tot_T_L_SD_LC_nn')
read_df54 = pd.read_excel(excel_file, sheet_name='Tot_T_E_SD_LC_nn')
Tot_TC_HRL2_nn = read_df1['Tot_TC_HRL2_nn'].values
Tot_T_L_HRL2_nn = read_df2['Tot_T_L_HRL2_nn'].values
Tot_T_E_HRL2_nn = read_df3['Tot_T_E_HRL2_nn'].values
No_of_Ser_L_Fail_HRL2_nn = read_df4['No_of_Ser_L_Fail_HRL2_nn'].values
No_of_Soj_L_Fail_HRL2_nn = read_df5['No_of_Soj_L_Fail_HRL2_nn'].values
No_of_Ser_H_Req_HRL2_nn = read_df6['No_of_Ser_H_Req_HRL2_nn'].values
Tot_TC_SD_HRL2_nn = read_df7['Tot_TC_SD_HRL2_nn'].values
Tot_T_L_SD_HRL2_nn = read_df8['Tot_T_L_SD_HRL2_nn'].values
Tot_T_E_SD_HRL2_nn = read_df9['Tot_T_E_SD_HRL2_nn'].values
Tot_TC_HRL1_nn = read_df10['Tot_TC_HRL1_nn'].values
Tot_T_L_HRL1_nn = read_df11['Tot_T_L_HRL1_nn'].values
Tot_T_E_HRL1_nn = read_df12['Tot_T_E_HRL1_nn'].values
No_of_Ser_L_Fail_HRL1_nn = read_df13['No_of_Ser_L_Fail_HRL1_nn'].values
No_of_Soj_L_Fail_HRL1_nn = read_df14['No_of_Soj_L_Fail_HRL1_nn'].values
No_of_Ser_H_Req_HRL1_nn = read_df15['No_of_Ser_H_Req_HRL1_nn'].values
Tot_TC_SD_HRL1_nn = read_df16['Tot_TC_SD_HRL1_nn'].values
Tot_T_L_SD_HRL1_nn = read_df17['Tot_T_L_SD_HRL1_nn'].values
Tot_T_E_SD_HRL1_nn = read_df18['Tot_T_E_SD_HRL1_nn'].values
Tot_TC_HRL_nn = read_df19['Tot_TC_HRL_nn'].values
Tot_T_L_HRL_nn = read_df20['Tot_T_L_HRL_nn'].values
Tot_T_E_HRL_nn = read_df21['Tot_T_E_HRL_nn'].values
No_of_Ser_L_Fail_HRL_nn = read_df22['No_of_Ser_L_Fail_HRL_nn'].values
No_of_Soj_L_Fail_HRL_nn = read_df23['No_of_Soj_L_Fail_HRL_nn'].values
No_of_Ser_H_Req_HRL_nn = read_df24['No_of_Ser_H_Req_HRL_nn'].values
Tot_TC_SD_HRL_nn = read_df25['Tot_TC_SD_HRL_nn'].values
Tot_T_L_SD_HRL_nn = read_df26['Tot_T_L_SD_HRL_nn'].values
Tot_T_E_SD_HRL_nn = read_df27['Tot_T_E_SD_HRL_nn'].values
Tot_TC_DA_nn = read_df28['Tot_TC_DA_nn'].values
Tot_T_L_DA_nn = read_df29['Tot_T_L_DA_nn'].values
Tot_T_E_DA_nn = read_df30['Tot_T_E_DA_nn'].values
No_of_Ser_L_Fail_DA_nn = read_df31['No_of_Ser_L_Fail_DA_nn'].values
No_of_Soj_L_Fail_DA_nn = read_df32['No_of_Soj_L_Fail_DA_nn'].values
No_of_Ser_H_Req_DA_nn = read_df33['No_of_Ser_H_Req_DA_nn'].values
Tot_TC_SD_DA_nn = read_df34['Tot_TC_SD_DA_nn'].values
Tot_T_L_SD_DA_nn = read_df35['Tot_T_L_SD_DA_nn'].values
Tot_T_E_SD_DA_nn = read_df36['Tot_T_E_SD_DA_nn'].values
Tot_TC_RA_nn = read_df37['Tot_TC_RA_nn'].values
Tot_T_L_RA_nn = read_df38['Tot_T_L_RA_nn'].values
Tot_T_E_RA_nn = read_df39['Tot_T_E_RA_nn'].values
No_of_Ser_L_Fail_RA_nn = read_df40['No_of_Ser_L_Fail_RA_nn'].values
No_of_Soj_L_Fail_RA_nn = read_df41['No_of_Soj_L_Fail_RA_nn'].values
No_of_Ser_H_Req_RA_nn = read_df42['No_of_Ser_H_Req_RA_nn'].values
Tot_TC_SD_RA_nn = read_df43['Tot_TC_SD_RA_nn'].values
Tot_T_L_SD_RA_nn = read_df44['Tot_T_L_SD_RA_nn'].values
Tot_T_E_SD_RA_nn = read_df45['Tot_T_E_SD_RA_nn'].values
Tot_TC_LC_nn = read_df46['Tot_TC_LC_nn'].values
Tot_T_L_LC_nn = read_df47['Tot_T_L_LC_nn'].values
Tot_T_E_LC_nn = read_df48['Tot_T_E_LC_nn'].values
No_of_Ser_L_Fail_LC_nn = read_df49['No_of_Ser_L_Fail_LC_nn'].values
No_of_Soj_L_Fail_LC_nn = read_df50['No_of_Soj_L_Fail_LC_nn'].values
No_of_Ser_H_Req_LC_nn = read_df51['No_of_Ser_H_Req_LC_nn'].values
Tot_TC_SD_LC_nn = read_df52['Tot_TC_SD_LC_nn'].values
Tot_T_L_SD_LC_nn = read_df53['Tot_T_L_SD_LC_nn'].values
Tot_T_E_SD_LC_nn = read_df54['Tot_T_E_SD_LC_nn'].values



In [ ]:
plt.plot(Users, Tot_TC_HRL_nn, color="green", marker="*")
plt.errorbar(Users, Tot_TC_HRL_nn, yerr=Tot_TC_SD_HRL_nn, label="HRL ($\\gamma_1, \\gamma_2$ = 0.5)", color="green", marker="*")

plt.plot(Users, Tot_TC_HRL1_nn, color="purple", marker="d")
plt.errorbar(Users, Tot_TC_HRL1_nn, yerr=Tot_TC_SD_HRL1_nn, label="HRL1 ($\\gamma_1=0.25, \\gamma_2$ = 0.75)", color="purple", marker="d")

plt.plot(Users, Tot_TC_HRL2_nn, color="orange", marker="h")
plt.errorbar(Users, Tot_TC_HRL2_nn, yerr=Tot_TC_SD_HRL2_nn, label="HRL2 ($\\gamma_1=0.75, \\gamma_2$ = 0.25)", color="orange", marker="h")

plt.plot(Users, Tot_TC_DA_nn, color="black", marker="o")
plt.errorbar(Users, Tot_TC_DA_nn, yerr=Tot_TC_SD_DA_nn, label="DM", color="black", marker="o")

plt.plot(Users, Tot_TC_RA_nn, color="blue", marker="^")
plt.errorbar(Users, Tot_TC_RA_nn, yerr=Tot_TC_SD_RA_nn, label="RM", color="blue", marker="^")

plt.plot(Users, Tot_TC_LC_nn, color="red", marker="s")
plt.errorbar(Users, Tot_TC_LC_nn, yerr=Tot_TC_SD_LC_nn, label="LC", color="red", marker="s")

plt.grid()
plt.xlabel('No. of Users')
plt.ylabel('Avg. Cost')
plt.legend()
plt.xlim([Users[0], Users[len(Users)-1]])
plt.savefig('TC_r', dpi=600)
plt.show()


In [ ]:
plt.plot(Users, Tot_T_L_HRL_nn, color="green", marker="*")
plt.plot(Users, Tot_T_L_DA_nn, color="black", marker="o")
plt.plot(Users, Tot_T_L_RA_nn, color="blue", marker="^")
plt.plot(Users, Tot_T_L_LC_nn, color="red", marker="s")
plt.plot(Users, Req_LC, label="Req. Latency", color="cyan", marker="p")

plt.errorbar(Users, Tot_T_L_HRL_nn, yerr=Tot_T_L_SD_HRL_nn, label="HRL", color="green", marker="*")
plt.errorbar(Users, Tot_T_L_DA_nn, yerr=Tot_T_L_SD_DA_nn, label="DM", color="black", marker="o")
plt.errorbar(Users, Tot_T_L_RA_nn, yerr=Tot_T_L_SD_RA_nn, label="RM", color="blue", marker="^")
plt.errorbar(Users, Tot_T_L_LC_nn, yerr=Tot_T_L_SD_LC_nn, label="LC", color="red", marker="s")

plt.plot(Users, Tot_T_L_HRL1_nn, color="purple", marker="d")
plt.errorbar(Users, Tot_T_L_HRL1_nn, yerr=Tot_T_L_SD_HRL1_nn, label="HRL1 ($\\gamma_1=0.25, \\gamma_2$ = 0.75)", color="purple", marker="d")

plt.plot(Users, Tot_T_L_HRL2_nn, color="orange", marker="h")
plt.errorbar(Users, Tot_T_L_HRL2_nn, yerr=Tot_T_L_SD_HRL2_nn, label="HRL2 ($\\gamma_1=0.75, \\gamma_2$ = 0.25)", color="orange", marker="h")

plt.grid()
plt.xlabel('No. of Users')
plt.ylabel('Avg. Latency Cost [Sec]')
plt.legend()
plt.xlim([Users[0], Users[len(Users)-1]])
plt.savefig('LC_r', dpi=600)
plt.show()


In [ ]:
plt.plot(Users, Tot_T_E_HRL_nn, color="green", marker="*")
plt.errorbar(Users, Tot_T_E_HRL_nn, yerr=Tot_T_E_SD_HRL_nn, label="HRL", color="green", marker="*")

plt.plot(Users, Tot_T_E_DA_nn, color="black", marker="o")
plt.errorbar(Users, Tot_T_E_DA_nn, yerr=Tot_T_E_SD_DA_nn, label="DM", color="black", marker="o")

plt.plot(Users, Tot_T_E_RA_nn, color="blue", marker="^")
plt.errorbar(Users, Tot_T_E_RA_nn, yerr=Tot_T_E_SD_RA_nn, label="RM", color="blue", marker="^")

plt.plot(Users, Tot_T_E_LC_nn, color="red", marker="s")
plt.errorbar(Users, Tot_T_E_LC_nn, yerr=Tot_T_E_SD_LC_nn, label="LC", color="red", marker="s")

plt.plot(Users, Tot_T_E_HRL1_nn, color="purple", marker="d")
plt.errorbar(Users, Tot_T_E_HRL1_nn, yerr=Tot_T_E_SD_HRL1_nn, label="HRL ($\\gamma_1=0.25, \\gamma_2$ = 0.75)", color="purple", marker="d")

plt.plot(Users, Tot_T_E_HRL2_nn, color="orange", marker="h")
plt.errorbar(Users, Tot_T_E_HRL2_nn, yerr=Tot_T_E_SD_HRL2_nn, label="HRL ($\\gamma_1=0.75, \\gamma_2$ = 0.25)", color="orange", marker="h")

plt.grid()
plt.xlabel('No. of Users')
plt.ylabel('Avg. Energy Cost [J]')
plt.legend()
plt.xlim([Users[0], Users[len(Users)-1]])
plt.savefig('EC_r', dpi=600)
plt.show()


In [ ]:
plt.plot(Users, No_of_Soj_L_Fail_HRL_nn, label="HRL", color="green", marker="*")
plt.plot(Users, No_of_Soj_L_Fail_DA_nn, label="DM", color="black", marker="o")
plt.plot(Users, No_of_Soj_L_Fail_RA_nn, label="RM", color="blue", marker="^")
plt.plot(Users, No_of_Soj_L_Fail_LC_nn, label="LC", color="red", marker="s")

plt.plot(Users, No_of_Soj_L_Fail_HRL1_nn, label="HRL ($\\gamma_1=0.25, \\gamma_2$ = 0.75)", color="purple", marker="*")
plt.plot(Users, No_of_Soj_L_Fail_HRL2_nn, label="HRL ($\\gamma_1=0.75, \\gamma_2$ = 0.25)", color="orange", marker="*")

plt.grid()
plt.xlabel('No. of Users')
plt.ylabel('Avg. No. of Handovers (Coverage Node)')
plt.legend()
plt.xlim([Users[0], Users[len(Users)-1]])
plt.savefig('CH_r', dpi=600)
plt.show()


In [ ]:
plt.plot(Users, No_of_Ser_L_Fail_HRL_nn, label="HRL", color="green", marker="*")
plt.plot(Users, No_of_Ser_L_Fail_DA_nn, label="DM", color="black", marker="o")
plt.plot(Users, No_of_Ser_L_Fail_RA_nn, label="RM", color="blue", marker="^")
plt.plot(Users, No_of_Ser_L_Fail_LC_nn, label="LC", color="red", marker="s")

plt.plot(Users, No_of_Ser_L_Fail_HRL1_nn, label="HRL ($\\gamma_1=0.25, \\gamma_2$ = 0.75)", color="purple", marker="*")
plt.plot(Users, No_of_Ser_L_Fail_HRL2_nn, label="HRL ($\\gamma_1=0.75, \\gamma_2$ = 0.25)", color="orange", marker="*")

plt.grid()
plt.xlabel('No. of Users')
plt.ylabel('Service Latency Failures (Avg)')
plt.legend()
plt.xlim([Users[0], Users[len(Users)-1]])
plt.savefig('SF_r', dpi=600)
plt.show()


In [ ]:
plt.plot(Users, No_of_Ser_H_Req_HRL_nn, label="HRL", color="green", marker="*")
plt.plot(Users, No_of_Ser_H_Req_DA_nn, label="DM", color="black", marker="o")
plt.plot(Users, No_of_Ser_H_Req_RA_nn, label="RM", color="blue", marker="^")
plt.plot(Users, No_of_Ser_H_Req_LC_nn, label="LC", color="red", marker="s")

plt.plot(Users, No_of_Ser_H_Req_HRL1_nn, label="HRL ($\\gamma_1=0.25, \\gamma_2$ = 0.75)", color="purple", marker="*")
plt.plot(Users, No_of_Ser_H_Req_HRL2_nn, label="HRL ($\\gamma_1=0.75, \\gamma_2$ = 0.25)", color="orange", marker="*")

plt.grid()
plt.xlabel('No. of Users')
plt.ylabel('Avg. No. of Handovers (Service)')
plt.legend()
plt.xlim([Users[0], Users[len(Users)-1]])
plt.savefig('SH_r', dpi=600)
plt.show()
